# Install Requirements

In [ ]:
%pip install -r requirements.txt

# Dependencies

In [23]:
import pandas as pd
import numpy as np
import os, requests, json, csv
import psycopg2
from dotenv import load_dotenv

In [24]:
from pydantic import BaseModel, Field
from openai import OpenAI

# Initialize Ontology Translator

In [26]:
import psycopg2
import os
from contextlib import closing

def load_ontology_from_db() -> dict:
    ontology_data = {
        "uberon-tissue": {},
        "cl-cell": {},
        "ensembl-gene": {}
    }

    table_mappings = [
        ("TISSUE", "UBERON", "tissueName", "uberon-tissue"),
        ("CELL", "CL", "cellName", "cl-cell"),
        ("GENE", "ENSEMBL", "geneName", "ensembl-gene")
    ]

    query = " UNION ALL ".join(
        f"SELECT '{json_key}' AS type, {key_col} AS key, {value_col} AS value FROM {table}"
        for table, key_col, value_col, json_key in table_mappings
    )

    with closing(psycopg2.connect(os.getenv("DATABASE_URL"))) as conn, conn.cursor() as cur:
        cur.execute(query)
        for row in cur.fetchall():
            json_key, key, value = row
            ontology_data[json_key][key] = value

    return ontology_data

In [28]:
class OntologyTranslator:
    _ontology_dict = None  # Class-level cache

    @classmethod
    def _load_ontology(cls):
        if cls._ontology_dict is None:
            cls._ontology_dict = load_ontology_from_db()

    @classmethod
    def tissue(cls, value: str, rev: bool = False) -> str | None:
        cls._load_ontology()
        table = cls._ontology_dict.get("uberon-tissue", {})
        return (
            next((k for k, v in table.items() if v.lower() == value.lower()), None)
            if rev else table.get(value)
        )

    @classmethod
    def cell(cls, value: str, rev: bool = False) -> str | None:
        cls._load_ontology()
        table = cls._ontology_dict.get("cl-cell", {})
        return (
            next((k for k, v in table.items() if v.lower() == value.lower()), None)
            if rev else table.get(value)
        )

    @classmethod
    def gene(cls, value: str, rev: bool = False) -> str | None:
        cls._load_ontology()
        table = cls._ontology_dict.get("ensembl-gene", {})
        return (
            next((k for k, v in table.items() if v.lower() == value.lower()), None)
            if rev else table.get(value)
        )
    
ontl = OntologyTranslator

# Load CSV

In [29]:
def csv_read(file_path):
    with open(file_path, 'r') as file:
        sample = file.read(1024) 
        file.seek(0)
        detected_delimiter = csv.Sniffer().sniff(sample).delimiter    
    # Read the file with the detected delimiter
    df = pd.read_csv(file_path, sep=detected_delimiter)
    return df

In [30]:
file_path = "cluster0.csv"
df = csv_read(file_path)

In [31]:
df

,myAUC,avg_diff,power,avg_log2FC,pct.1,pct.2,Cluster,Gene
0,0.939,0.918256,0.878,1.351082,1.000,0.996,0,KRT14
1,0.855,0.713125,0.710,1.219866,1.000,0.848,0,IFI27
2,0.755,0.507771,0.510,1.248768,0.875,0.509,0,LGALS3BP
3,0.741,0.476354,0.482,1.234963,0.848,0.460,0,RBM42
4,0.703,0.363869,0.406,1.289658,0.694,0.300,0,GAMT
...,...,...,...,...,...,...,...,...
128,0.839,0.837200,0.678,1.736036,0.942,0.438,25,COL6A3
129,0.738,0.427259,0.476,1.844166,0.626,0.142,25,COL11A1
130,0.724,0.189142,0.448,0.306940,1.000,0.828,26,SPARC
131,0.703,0.282657,0.406,0.431395,1.000,0.998,26,MT-CYB


# Preprocess Data

## Group Genes by Clusters

In [32]:
def extract_clusters(df):
    df.columns = df.columns.str.lower()
    # Ensure the column names are correct
    if "cluster" not in df.columns or "gene" not in df.columns:
        raise ValueError("The file must contain 'Cluster' and 'Gene' columns")

    # Group the data by clusters and pad genes to ensure columns have equal lengths
    grouped = df.groupby("cluster")["gene"].apply(list)

    # Create a DataFrame where each column is a cluster and rows contain the genes
    max_length = max(grouped.apply(len))  # Determine the maximum number of genes in a cluster
    return pd.DataFrame({cluster: genes + [None] * (max_length - len(genes)) for cluster, genes in grouped.items()})

In [33]:
clustered_df = extract_clusters(df)

clustered_df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,KRT14,TXNIP,LY6D,S100A8,TIMP1,PERP,TINAGL1,LTF,KRT16,KRT14,...,IGHG4,THRSP,IGKC,SPRR2D,IGHG4,SAA1,LOR,S100A8,COL3A1,SPARC
1,IFI27,ATP1B3,KRT16,S100A7,APOD,GPNMB,IGFBP6,CCL27,SBSN,LY6D,...,IGHG1,PECR,IGHG3,SPRR1A,IGKC,AWAT2,LCE1B,S100A7,COL1A1,MT-CYB
2,LGALS3BP,MAF,HIST1H1C,SPRR1B,MMP3,GJB6,MMP28,FOSB,LGALS7,LGALS7,...,JCHAIN,HSD3B1,IGHA1,IL36RN,C1QA,ACSBG1,WFDC12,CSTA,DCN,MT-ND2
3,RBM42,MMP13,LSR,KRTDAP,COMP,CLDN1,CENPW,ATF3,KRTDAP,IFI27,...,IGFBP6,FABP7,IGLC2,LCE3E,MYL9,CYP4F8,LCE1A,S100A7A,COL6A3,None
4,GAMT,PLAU,TMEM79,PI3,CCL19,TNFSF10,CCNB2,AC103591.3,HIST1H1C,TAGLN2,...,SULF1,ACSM6,IGHG2,SLC6A14,IGHG1,GLDC,LCE1E,SPP1,COL11A1,None


## Translate Genes into Ensembl ID

In [34]:
def translate_genes(clustered_df):
    # Translate gene names to Ensembl IDs in the clustered DataFrame
    return clustered_df.apply(lambda col: col.map(lambda gene: ontl.gene(gene, rev=True) if pd.notna(gene) else None))

In [35]:
ensembl_df = translate_genes(clustered_df)
ensembl_df

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,ENSG00000186847,ENSG00000265972,ENSG00000167656,ENSG00000143546,ENSG00000102265,ENSG00000112378,ENSG00000142910,ENSG00000012223,ENSG00000186832,ENSG00000186847,...,ENSG00000211892,ENSG00000151365,ENSG00000211592,ENSG00000163216,ENSG00000211892,ENSG00000173432,None,ENSG00000143546,ENSG00000168542,ENSG00000113140
1,ENSG00000165949,ENSG00000069849,ENSG00000186832,ENSG00000143556,ENSG00000189058,ENSG00000136235,ENSG00000167779,ENSG00000213927,ENSG00000189001,ENSG00000167656,...,ENSG00000211896,ENSG00000115425,ENSG00000211897,ENSG00000169474,ENSG00000211592,ENSG00000147160,ENSG00000196734,ENSG00000143556,ENSG00000108821,ENSG00000198727
2,ENSG00000108679,ENSG00000178573,None,ENSG00000169469,ENSG00000149968,ENSG00000121742,ENSG00000271447,ENSG00000125740,ENSG00000205076,ENSG00000205076,...,ENSG00000132465,ENSG00000203857,ENSG00000211895,ENSG00000136695,ENSG00000173372,ENSG00000103740,ENSG00000168703,ENSG00000121552,ENSG00000011465,ENSG00000198763
3,ENSG00000126254,ENSG00000137745,ENSG00000105699,ENSG00000188508,ENSG00000105664,ENSG00000163347,ENSG00000203760,ENSG00000162772,ENSG00000188508,ENSG00000165949,...,ENSG00000167779,ENSG00000164434,ENSG00000211677,ENSG00000185966,ENSG00000101335,ENSG00000186526,ENSG00000186844,ENSG00000184330,ENSG00000163359,None
4,ENSG00000130005,ENSG00000122861,ENSG00000163472,ENSG00000124102,ENSG00000172724,ENSG00000121858,ENSG00000157456,None,None,ENSG00000158710,...,ENSG00000137573,ENSG00000173124,ENSG00000211893,ENSG00000268104,ENSG00000211896,ENSG00000178445,ENSG00000186226,ENSG00000118785,ENSG00000060718,None


# Statistical Analysis with CellxGene's Gene Expression Data

## Function Initialization

Function to fetch gene expression data

In [36]:
def fetch_expression_data(cluster, ensembl_ids):
    """
    Fetches gene expression data from the Cellxgene API for a given cluster and list of Ensembl IDs.
    
    :param cluster: The cluster name.
    :param ensembl_ids: List of Ensembl IDs for the cluster.
    :return: JSON response from the API or None if an error occurred.
    """

    payload = {
        "filter": {
            "dataset_ids": [],
            "development_stage_ontology_term_ids": [],
            "disease_ontology_term_ids": [],
            "gene_ontology_term_ids": ensembl_ids,
            "organism_ontology_term_id": "NCBITaxon:9606",
            "self_reported_ethnicity_ontology_term_ids": [],
            "sex_ontology_term_ids": [],
            "publication_citations": [],
        },
        "is_rollup": True
    }

    # URL for the POST request
    API_URL = "https://api.cellxgene.cziscience.com/wmg/v2/query"

    try:
        response = requests.post(API_URL, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors
        return response.json()['expression_summary']
    except requests.RequestException as e:
        print(f"Error fetching data for Cluster {cluster}: {e}")
        return None

Function to convert the data to a DataFrame

In [37]:
def expression_data_to_df(data):
    flattened_data = []
    for gene_id, anatomical_structures in data.items():
        for anatomical_id, cell_types in anatomical_structures.items():
            for cell_type_id, aggregated_data in cell_types.items():
                metrics = aggregated_data['aggregated']
                flattened_data.append({
                    'gene': gene_id,
                    'tissue': anatomical_id,
                    'cell': cell_type_id,
                    'expression': metrics.get('me', None),
                    'cell count': metrics.get('n', None),
                    'cell percentage': metrics.get('pc', None),
                    'tissue composition': metrics.get('tpc', None)
                })
    return pd.DataFrame(flattened_data)

Function to filter the data

In [38]:
def filter_expression_data(response_df, TARGET_UBERON_ID):
    tissue_df = response_df[response_df['tissue'] == TARGET_UBERON_ID].drop(columns=['tissue'])
    return tissue_df[~tissue_df['cell'].isin(['tissue_stats', 'CL:0000000'])]

Function to translate the ontology terms

In [81]:
def translate_ontology(df):
    df['cell'] = df['cell'].apply(lambda x: ontl.cell(x)).fillna(df['cell'])
    df['gene'] = df['gene'].apply(lambda x: ontl.gene(x)).fillna(df['gene'])

    return df

In [59]:
def calculate_cell_score(
    results,
    w_e=1.0, w_p=1.5, w_ct=2.5,             # base score weights
    w_m=7.0, w_f=3.0,                       # mean vs flatness weights
    alpha=1.0, beta=1.0, gamma=1.0,         # flatness breakdown
    w_c=1.0                                 # count weight
):
    cell_df = results.copy()

    # Calculate total cell count per cell
    sum_cell_count = cell_df.groupby('cell')['cell count'].sum().reset_index()
    sum_cell_count.columns = ['cell', 'total cell count']
    cell_df = cell_df.merge(sum_cell_count, on='cell')

    # Adaptive cutoff: max(μ - 0.5σ, min(total count))
    mu = cell_df['total cell count'].mean()
    sigma = cell_df['total cell count'].std()
    minimum = cell_df['total cell count'].min()
    min_count = max(mu - 0.5 * sigma, minimum)
    cell_df = cell_df[cell_df['total cell count'] >= min_count]

    # Normalize expression and cell count
    norm_expr = (cell_df['expression'] - cell_df['expression'].min()) / (cell_df['expression'].max() - cell_df['expression'].min())
    norm_count = (cell_df['cell count'] - cell_df['cell count'].min()) / (cell_df['cell count'].max() - cell_df['cell count'].min())

    # Compute base score
    cell_df['score'] = (
        w_e * norm_expr +
        w_p * cell_df['cell percentage'] +
        w_ct * norm_count
    )

    # Compute flatness components
    std_scores = cell_df.groupby('cell')['score'].std().reset_index().rename(columns={'score': 'std'})
    mean_scores = cell_df.groupby('cell')['score'].mean().reset_index().rename(columns={'score': 'mean'})
    kurtosis_scores = cell_df.groupby('cell')['score'].apply(pd.Series.kurt).reset_index(name='kurtosis')

    cell_df = cell_df.merge(std_scores, on='cell')
    cell_df = cell_df.merge(mean_scores, on='cell')
    cell_df = cell_df.merge(kurtosis_scores, on='cell')

    cell_df['CV'] = cell_df['std'] / (cell_df['mean'].replace(0, np.nan))

    # Normalize flatness metrics
    for col in ['std', 'CV', 'kurtosis', 'mean']:
        col_min, col_max = cell_df[col].min(), cell_df[col].max()
        cell_df[f'{col} norm'] = (cell_df[col] - col_min) / (col_max - col_min + 1e-9)

    # Stability (flatness) score
    stability = (
        alpha * (1 - cell_df['std norm'].fillna(0)) +
        beta * (1 - cell_df['CV norm'].fillna(0)) +
        gamma * (1 - cell_df['kurtosis norm'].fillna(0))
    )
    flatness_weight = stability / (alpha + beta + gamma)

    # Count-based log-scaled soft weight
    cell_counts = cell_df['cell'].value_counts().reset_index()
    cell_counts.columns = ['cell', 'count']
    cell_df = cell_df.merge(cell_counts, on='cell')

    log_scaled_count = np.log1p(cell_df['count']) / np.log1p(cell_df['count'].max())

    # Final cell score
    cell_df['cell score'] = (
        (w_m * cell_df['mean norm'] + w_f * flatness_weight) *
        (1 + w_c * log_scaled_count)
    )

    return cell_df.sort_values(by=['cell score', 'cell', 'gene'], ascending=[False, False, True]).reset_index(drop=True)


In [78]:
def run_data_processing(cluster, ensembl_ids, target_uberon_id, n = 10):
    try:
        data = fetch_expression_data(cluster, ensembl_ids)

        response_df = expression_data_to_df(data)

        filtered_df = filter_expression_data(response_df, target_uberon_id)

        ranked_df = calculate_cell_score(filtered_df)

        unique_cells = ranked_df['cell'].dropna().unique()[:n]
        to_translate = ranked_df[ranked_df['cell'].isin(unique_cells)].copy()
        translated_df = translate_ontology(to_translate)

        return translated_df

    except Exception as e:
        print(f"An error occurred during the pipeline execution: {e}")
        return pd.DataFrame()

In [83]:
def run_data_processing(cluster, ensembl_ids, target_uberon_id, n = 10):
    try:
        data = fetch_expression_data(cluster, ensembl_ids)

        response_df = expression_data_to_df(data)

        filtered_df = filter_expression_data(response_df, target_uberon_id)

        translated_df = translate_ontology(filtered_df)

        ranked_df = calculate_cell_score(translated_df)

        return ranked_df

    except Exception as e:
        print(f"An error occurred during the pipeline execution: {e}")
        return pd.DataFrame()

In [61]:
def main_data_analysis(ensembl_df, target_uberon_id):
    results = {}

    # Iterate over each cluster (column) in the DataFrame
    for cluster in ensembl_df.columns:
        # Extract Ensembl IDs for the current cluster, dropping any NaN values
        ensembl_ids = ensembl_df[cluster].dropna().tolist()
        
        if not ensembl_ids:
            continue
        
        # Run the expression pipeline for the current cluster
        result_df = run_data_processing(cluster, ensembl_ids, target_uberon_id)
        
        # Store the result in the dictionary
        results[cluster] = result_df.reset_index(drop=True)

    return results

Function to Display the Results

In [44]:
from IPython.display import Markdown, display
def display_dataframe_as_markdown(df):
    markdown_table = df.to_markdown(index=False)
    display(Markdown(markdown_table))

## Run the Section

In [45]:
# Specify the UBERON ID to filter
TARGET_UBERON_ID = "UBERON:0002097"

In [84]:
# Run the main pipeline
results = main_data_analysis(ensembl_df, TARGET_UBERON_ID)

In [86]:
display_dataframe_as_markdown(results[1])

| gene   | cell                                                    |   expression |   cell count |   cell percentage |   tissue composition |   total cell count |       score |      std |     mean |   kurtosis |       CV |   std norm |   CV norm |   kurtosis norm |   mean norm |   count |   cell score |
|:-------|:--------------------------------------------------------|-------------:|-------------:|------------------:|---------------------:|-------------------:|------------:|---------:|---------:|-----------:|---------:|-----------:|----------:|----------------:|------------:|--------:|-------------:|
| ATP1B3 | hematopoietic cell                                      |      1.96266 |        69209 |       0.224443    |          0.0847753   |             188775 | 2.57641     | 1.2642   | 1.58521  |  -2.42958  | 0.797495 | 0.775047   | 0.491174  |       0.292183  |   1         |       5 |     16.8832  |
| MAF    | hematopoietic cell                                      |      2.02133 |        33625 |       0.11267     |          0.0411878   |             188775 | 1.49561     | 1.2642   | 1.58521  |  -2.42958  | 0.797495 | 0.775047   | 0.491174  |       0.292183  |   1         |       5 |     16.8832  |
| MMP13  | hematopoietic cell                                      |      1.68807 |         1792 |       0.00738753  |          0.00219505  |             188775 | 0.239547    | 1.2642   | 1.58521  |  -2.42958  | 0.797495 | 0.775047   | 0.491174  |       0.292183  |   1         |       5 |     16.8832  |
| PLAU   | hematopoietic cell                                      |      1.90397 |         4615 |       0.0159959   |          0.00565299  |             188775 | 0.485719    | 1.2642   | 1.58521  |  -2.42958  | 0.797495 | 0.775047   | 0.491174  |       0.292183  |   1         |       5 |     16.8832  |
| TXNIP  | hematopoietic cell                                      |      2.26996 |        79534 |       0.258402    |          0.0974225   |             188775 | 3.12878     | 1.2642   | 1.58521  |  -2.42958  | 0.797495 | 0.775047   | 0.491174  |       0.292183  |   1         |       5 |     16.8832  |
| ATP1B3 | leukocyte                                               |      1.96448 |        68687 |       0.225378    |          0.0841359   |             187351 | 2.56511     | 1.25528  | 1.57875  |  -2.43961  | 0.795111 | 0.768526   | 0.488119  |       0.29122   |   0.994955  |       5 |     16.8336  |
| MAF    | leukocyte                                               |      2.02167 |        33586 |       0.113263    |          0.0411401   |             187351 | 1.49571     | 1.25528  | 1.57875  |  -2.43961  | 0.795111 | 0.768526   | 0.488119  |       0.29122   |   0.994955  |       5 |     16.8336  |
| MMP13  | leukocyte                                               |      1.68807 |         1792 |       0.00738753  |          0.00219505  |             187351 | 0.239547    | 1.25528  | 1.57875  |  -2.43961  | 0.795111 | 0.768526   | 0.488119  |       0.29122   |   0.994955  |       5 |     16.8336  |
| PLAU   | leukocyte                                               |      1.90549 |         4588 |       0.0160711   |          0.00561992  |             187351 | 0.486216    | 1.25528  | 1.57875  |  -2.43961  | 0.795111 | 0.768526   | 0.488119  |       0.29122   |   0.994955  |       5 |     16.8336  |
| TXNIP  | leukocyte                                               |      2.2705  |        78698 |       0.258687    |          0.0963985   |             187351 | 3.10714     | 1.25528  | 1.57875  |  -2.43961  | 0.795111 | 0.768526   | 0.488119  |       0.29122   |   0.994955  |       5 |     16.8336  |
| ATP1B3 | mononuclear cell                                        |      1.96129 |        67295 |       0.224851    |          0.0824308   |             183535 | 2.5246      | 1.23811  | 1.55494  |  -2.4439   | 0.796244 | 0.755979   | 0.489571  |       0.290808  |   0.976389  |       5 |     16.5967  |
| MAF    | mononuclear cell                                        |      2.02028 |        33141 |       0.113865    |          0.040595    |             183535 | 1.48364     | 1.23811  | 1.55494  |  -2.4439   | 0.796244 | 0.755979   | 0.489571  |       0.290808  |   0.976389  |       5 |     16.5967  |
| MMP13  | mononuclear cell                                        |      1.68429 |         1757 |       0.00737532  |          0.00215218  |             183535 | 0.235834    | 1.23811  | 1.55494  |  -2.4439   | 0.796244 | 0.755979   | 0.489571  |       0.290808  |   0.976389  |       5 |     16.5967  |
| PLAU   | mononuclear cell                                        |      1.89698 |         4250 |       0.0151735   |          0.0052059   |             183535 | 0.469589    | 1.23811  | 1.55494  |  -2.4439   | 0.796244 | 0.755979   | 0.489571  |       0.290808  |   0.976389  |       5 |     16.5967  |
| TXNIP  | mononuclear cell                                        |      2.26864 |        77092 |       0.257616    |          0.0944313   |             183535 | 3.06104     | 1.23811  | 1.55494  |  -2.4439   | 0.796244 | 0.755979   | 0.489571  |       0.290808  |   0.976389  |       5 |     16.5967  |
| ATP1B3 | epithelial cell                                         |      2.03209 |        93064 |       0.516514    |          0.113996    |             172695 | 3.70597     | 1.57197  | 1.57007  |  -2.1698   | 1.00121  | 1          | 0.752262  |       0.31713   |   0.988191  |       5 |     15.6959  |
| MAF    | epithelial cell                                         |      1.75391 |         6982 |       0.0389975   |          0.00855237  |             172695 | 0.474393    | 1.57197  | 1.57007  |  -2.1698   | 1.00121  | 1          | 0.752262  |       0.31713   |   0.988191  |       5 |     15.6959  |
| MMP13  | epithelial cell                                         |      1.7066  |          448 |       0.00310482  |          0.000548763 |             172695 | 0.210537    | 1.57197  | 1.57007  |  -2.1698   | 1.00121  | 1          | 0.752262  |       0.31713   |   0.988191  |       5 |     15.6959  |
| PLAU   | epithelial cell                                         |      2.00531 |         7495 |       0.0417242   |          0.00918075  |             172695 | 0.675573    | 1.57197  | 1.57007  |  -2.1698   | 1.00121  | 1          | 0.752262  |       0.31713   |   0.988191  |       5 |     15.6959  |
| TXNIP  | epithelial cell                                         |      2.13605 |        64706 |       0.359125    |          0.0792595   |             172695 | 2.78389     | 1.57197  | 1.57007  |  -2.1698   | 1.00121  | 1          | 0.752262  |       0.31713   |   0.988191  |       5 |     15.6959  |
| ATP1B3 | squamous epithelial cell                                |      2.01621 |        70184 |       0.515551    |          0.0859696   |             134858 | 3.07831     | 1.31037  | 1.37216  |  -2.55305  | 0.954966 | 0.80879    | 0.692994  |       0.280326  |   0.833834  |       5 |     14.1095  |
| MAF    | squamous epithelial cell                                |      1.74131 |         6308 |       0.046728    |          0.00772677  |             134858 | 0.458694    | 1.31037  | 1.37216  |  -2.55305  | 0.954966 | 0.80879    | 0.692994  |       0.280326  |   0.833834  |       5 |     14.1095  |
| MMP13  | squamous epithelial cell                                |      1.69463 |          222 |       0.00209053  |          0.000271932 |             134858 | 0.194211    | 1.31037  | 1.37216  |  -2.55305  | 0.954966 | 0.80879    | 0.692994  |       0.280326  |   0.833834  |       5 |     14.1095  |
| PLAU   | squamous epithelial cell                                |      2.00848 |         6319 |       0.046494    |          0.00774025  |             134858 | 0.65345     | 1.31037  | 1.37216  |  -2.55305  | 0.954966 | 0.80879    | 0.692994  |       0.280326  |   0.833834  |       5 |     14.1095  |
| TXNIP  | squamous epithelial cell                                |      2.14417 |        51825 |       0.380694    |          0.0634813   |             134858 | 2.47614     | 1.31037  | 1.37216  |  -2.55305  | 0.954966 | 0.80879    | 0.692994  |       0.280326  |   0.833834  |       5 |     14.1095  |
| ATP1B3 | epidermal cell                                          |      2.01847 |        69933 |       0.514758    |          0.0856621   |             134246 | 3.07203     | 1.30765  | 1.37033  |  -2.55811  | 0.954257 | 0.806802   | 0.692085  |       0.27984   |   0.832407  |       5 |     14.0962  |
| MAF    | epidermal cell                                          |      1.74486 |         6227 |       0.0462232   |          0.00762756  |             134246 | 0.458353    | 1.30765  | 1.37033  |  -2.55811  | 0.954257 | 0.806802   | 0.692085  |       0.27984   |   0.832407  |       5 |     14.0962  |
| MMP13  | epidermal cell                                          |      1.69599 |          221 |       0.00209564  |          0.000270707 |             134246 | 0.195183    | 1.30765  | 1.37033  |  -2.55811  | 0.954257 | 0.806802   | 0.692085  |       0.27984   |   0.832407  |       5 |     14.0962  |
| PLAU   | epidermal cell                                          |      2.01094 |         6249 |       0.046128    |          0.0076545   |             134246 | 0.652813    | 1.30765  | 1.37033  |  -2.55811  | 0.954257 | 0.806802   | 0.692085  |       0.27984   |   0.832407  |       5 |     14.0962  |
| TXNIP  | epidermal cell                                          |      2.14951 |        51616 |       0.379932    |          0.0632253   |             134246 | 2.47327     | 1.30765  | 1.37033  |  -2.55811  | 0.954257 | 0.806802   | 0.692085  |       0.27984   |   0.832407  |       5 |     14.0962  |
| ATP1B3 | stratified squamous epithelial cell                     |      2.01708 |        69793 |       0.515511    |          0.0854906   |             133737 | 3.06838     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MAF    | stratified squamous epithelial cell                     |      1.73854 |         6151 |       0.0458189   |          0.00753446  |             133737 | 0.451097    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MMP13  | stratified squamous epithelial cell                     |      1.69599 |          221 |       0.00209564  |          0.000270707 |             133737 | 0.195183    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| PLAU   | stratified squamous epithelial cell                     |      2.01091 |         6246 |       0.0462071   |          0.00765083  |             133737 | 0.65283     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| TXNIP  | stratified squamous epithelial cell                     |      2.14454 |        51326 |       0.379109    |          0.0628701   |             133737 | 2.46063     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| ATP1B3 | stratified epithelial cell                              |      2.01708 |        69793 |       0.515511    |          0.0854906   |             133737 | 3.06838     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MAF    | stratified epithelial cell                              |      1.73854 |         6151 |       0.0458189   |          0.00753446  |             133737 | 0.451097    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MMP13  | stratified epithelial cell                              |      1.69599 |          221 |       0.00209564  |          0.000270707 |             133737 | 0.195183    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| PLAU   | stratified epithelial cell                              |      2.01091 |         6246 |       0.0462071   |          0.00765083  |             133737 | 0.65283     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| TXNIP  | stratified epithelial cell                              |      2.14454 |        51326 |       0.379109    |          0.0628701   |             133737 | 2.46063     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| ATP1B3 | keratinocyte                                            |      2.01708 |        69793 |       0.515511    |          0.0854906   |             133737 | 3.06838     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MAF    | keratinocyte                                            |      1.73854 |         6151 |       0.0458189   |          0.00753446  |             133737 | 0.451097    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MMP13  | keratinocyte                                            |      1.69599 |          221 |       0.00209564  |          0.000270707 |             133737 | 0.195183    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| PLAU   | keratinocyte                                            |      2.01091 |         6246 |       0.0462071   |          0.00765083  |             133737 | 0.65283     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| TXNIP  | keratinocyte                                            |      2.14454 |        51326 |       0.379109    |          0.0628701   |             133737 | 2.46063     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| ATP1B3 | CL:0000237                                              |      2.01708 |        69793 |       0.515511    |          0.0854906   |             133737 | 3.06838     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MAF    | CL:0000237                                              |      1.73854 |         6151 |       0.0458189   |          0.00753446  |             133737 | 0.451097    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| MMP13  | CL:0000237                                              |      1.69599 |          221 |       0.00209564  |          0.000270707 |             133737 | 0.195183    | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| PLAU   | CL:0000237                                              |      2.01091 |         6246 |       0.0462071   |          0.00765083  |             133737 | 0.65283     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| TXNIP  | CL:0000237                                              |      2.14454 |        51326 |       0.379109    |          0.0628701   |             133737 | 2.46063     | 1.30506  | 1.36562  |  -2.5368   | 0.955655 | 0.804915   | 0.693877  |       0.281887  |   0.828736  |       5 |     14.0409  |
| ATP1B3 | myeloid leukocyte                                       |      1.95002 |        28196 |       0.316088    |          0.0345378   |              67161 | 1.60291     | 0.568262 | 0.965862 |  -1.89727  | 0.588347 | 0.266383   | 0.223124  |       0.343302  |   0.516954  |       5 |     11.5717  |
| MAF    | myeloid leukocyte                                       |      2.16538 |        17607 |       0.197621    |          0.0215671   |              67161 | 1.29778     | 0.568262 | 0.965862 |  -1.89727  | 0.588347 | 0.266383   | 0.223124  |       0.343302  |   0.516954  |       5 |     11.5717  |
| MMP13  | myeloid leukocyte                                       |      1.74324 |          206 |       0.00236817  |          0.000252333 |              67161 | 0.229644    | 0.568262 | 0.965862 |  -1.89727  | 0.588347 | 0.266383   | 0.223124  |       0.343302  |   0.516954  |       5 |     11.5717  |
| PLAU   | myeloid leukocyte                                       |      1.92875 |         3943 |       0.0442308   |          0.00482985  |              67161 | 0.528094    | 0.568262 | 0.965862 |  -1.89727  | 0.588347 | 0.266383   | 0.223124  |       0.343302  |   0.516954  |       5 |     11.5717  |
| TXNIP  | myeloid leukocyte                                       |      2.01569 |        17209 |       0.19292     |          0.0210796   |              67161 | 1.17089     | 0.568262 | 0.965862 |  -1.89727  | 0.588347 | 0.266383   | 0.223124  |       0.343302  |   0.516954  |       5 |     11.5717  |
| ATP1B3 | myeloid cell                                            |      1.94823 |        28389 |       0.310018    |          0.0347742   |              67547 | 1.59768     | 0.566616 | 0.963757 |  -1.9083   | 0.587924 | 0.26518    | 0.222582  |       0.342243  |   0.515313  |       5 |     11.5544  |
| MAF    | myeloid cell                                            |      2.16491 |        17624 |       0.195555    |          0.0215879   |              67547 | 1.29479     | 0.566616 | 0.963757 |  -1.9083   | 0.587924 | 0.26518    | 0.222582  |       0.342243  |   0.515313  |       5 |     11.5544  |
| MMP13  | myeloid cell                                            |      1.74324 |          206 |       0.00236817  |          0.000252333 |              67547 | 0.229644    | 0.566616 | 0.963757 |  -1.9083   | 0.587924 | 0.26518    | 0.222582  |       0.342243  |   0.515313  |       5 |     11.5544  |
| PLAU   | myeloid cell                                            |      1.92748 |         3963 |       0.0434078   |          0.00485435  |              67547 | 0.526465    | 0.566616 | 0.963757 |  -1.9083   | 0.587924 | 0.26518    | 0.222582  |       0.342243  |   0.515313  |       5 |     11.5544  |
| TXNIP  | myeloid cell                                            |      2.01554 |        17365 |       0.189738    |          0.0212707   |              67547 | 1.1702      | 0.566616 | 0.963757 |  -1.9083   | 0.587924 | 0.26518    | 0.222582  |       0.342243  |   0.515313  |       5 |     11.5544  |
| ATP1B3 | lymphocyte                                              |      1.97974 |        37587 |       0.190901    |          0.0460409   |             115545 | 1.68907     | 1.08772  | 1.13886  |  -0.530505 | 0.955095 | 0.646055   | 0.69316   |       0.474555  |   0.651877  |       5 |     11.4987  |
| MAF    | lymphocyte                                              |      1.86568 |        15547 |       0.081938    |          0.0190438   |             115545 | 0.850387    | 1.08772  | 1.13886  |  -0.530505 | 0.955095 | 0.646055   | 0.69316   |       0.474555  |   0.651877  |       5 |     11.4987  |
| MMP13  | lymphocyte                                              |      1.6793  |         1497 |       0.0108069   |          0.0018337   |             115545 | 0.230363    | 1.08772  | 1.13886  |  -0.530505 | 0.955095 | 0.646055   | 0.69316   |       0.474555  |   0.651877  |       5 |     11.4987  |
| PLAU   | lymphocyte                                              |      1.66354 |          366 |       0.00205908  |          0.00044832  |             115545 | 0.175365    | 1.08772  | 1.13886  |  -0.530505 | 0.955095 | 0.646055   | 0.69316   |       0.474555  |   0.651877  |       5 |     11.4987  |
| TXNIP  | lymphocyte                                              |      2.34772 |        60548 |       0.3075      |          0.0741663   |             115545 | 2.7491      | 1.08772  | 1.13886  |  -0.530505 | 0.955095 | 0.646055   | 0.69316   |       0.474555  |   0.651877  |       5 |     11.4987  |
| ATP1B3 | phagocyte                                               |      1.93793 |        29708 |       0.290134    |          0.0363898   |              67990 | 1.59578     | 0.562836 | 0.929985 |  -1.66049  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| MAF    | phagocyte                                               |      2.15689 |        17594 |       0.173658    |          0.0215512   |              67990 | 1.25529     | 0.562836 | 0.929985 |  -1.66049  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| MMP13  | phagocyte                                               |      1.71298 |          260 |       0.00260772  |          0.000318478 |              67990 | 0.209392    | 0.562836 | 0.929985 |  -1.66049  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| PLAU   | phagocyte                                               |      1.91898 |         3884 |       0.0379501   |          0.00475758  |              67990 | 0.509962    | 0.562836 | 0.929985 |  -1.66049  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| TXNIP  | phagocyte                                               |      1.97919 |        16544 |       0.161645    |          0.020265    |              67990 | 1.0795      | 0.562836 | 0.929985 |  -1.66049  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| ATP1B3 | mononuclear phagocyte                                   |      1.93793 |        29708 |       0.290134    |          0.0363898   |              67990 | 1.59578     | 0.562836 | 0.929985 |  -1.66049  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| MAF    | mononuclear phagocyte                                   |      2.15689 |        17594 |       0.173658    |          0.0215512   |              67990 | 1.25529     | 0.562836 | 0.929985 |  -1.66049  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| MMP13  | mononuclear phagocyte                                   |      1.71298 |          260 |       0.00260772  |          0.000318478 |              67990 | 0.209392    | 0.562836 | 0.929985 |  -1.66049  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| PLAU   | mononuclear phagocyte                                   |      1.91898 |         3884 |       0.0379501   |          0.00475758  |              67990 | 0.509962    | 0.562836 | 0.929985 |  -1.66049  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| TXNIP  | mononuclear phagocyte                                   |      1.97919 |        16544 |       0.161645    |          0.020265    |              67990 | 1.0795      | 0.562836 | 0.929985 |  -1.66049  | 0.60521  | 0.262418   | 0.244736  |       0.366041  |   0.488973  |       5 |     11.0992  |
| ATP1B3 | T cell                                                  |      2.0015  |        33486 |       0.21714     |          0.0410176   |              97598 | 1.63413     | 0.983809 | 1.06541  |  -1.12645  | 0.923411 | 0.570109   | 0.652553  |       0.417325  |   0.594591  |       5 |     11.0443  |
| MAF    | T cell                                                  |      1.87653 |        14423 |       0.0935307   |          0.017667    |              97598 | 0.845492    | 0.983809 | 1.06541  |  -1.12645  | 0.923411 | 0.570109   | 0.652553  |       0.417325  |   0.594591  |       5 |     11.0443  |
| MMP13  | T cell                                                  |      1.67873 |          986 |       0.00923982  |          0.00120777  |              97598 | 0.213866    | 0.983809 | 1.06541  |  -1.12645  | 0.923411 | 0.570109   | 0.652553  |       0.417325  |   0.594591  |       5 |     11.0443  |
| PLAU   | T cell                                                  |      1.65596 |          243 |       0.00175342  |          0.000297655 |              97598 | 0.166076    | 0.983809 | 1.06541  |  -1.12645  | 0.923411 | 0.570109   | 0.652553  |       0.417325  |   0.594591  |       5 |     11.0443  |
| TXNIP  | T cell                                                  |      2.39302 |        48460 |       0.314216    |          0.0593595   |              97598 | 2.46748     | 0.983809 | 1.06541  |  -1.12645  | 0.923411 | 0.570109   | 0.652553  |       0.417325  |   0.594591  |       5 |     11.0443  |
| ATP1B3 | professional antigen presenting cell                    |      1.94512 |        28387 |       0.29964     |          0.0347717   |              64385 | 1.57979     | 0.555087 | 0.918554 |  -1.67696  | 0.604306 | 0.256754   | 0.243577  |       0.364459  |   0.480057  |       5 |     10.9912  |
| MAF    | professional antigen presenting cell                    |      2.16865 |        17053 |       0.182514    |          0.0208885   |              64385 | 1.26262     | 0.555087 | 0.918554 |  -1.67696  | 0.604306 | 0.256754   | 0.243577  |       0.364459  |   0.480057  |       5 |     10.9912  |
| MMP13  | professional antigen presenting cell                    |      1.71997 |          237 |       0.00260927  |          0.000290305 |              64385 | 0.213872    | 0.555087 | 0.918554 |  -1.67696  | 0.604306 | 0.256754   | 0.243577  |       0.364459  |   0.480057  |       5 |     10.9912  |
| PLAU   | professional antigen presenting cell                    |      1.92183 |         3685 |       0.0390426   |          0.00451382  |              64385 | 0.508333    | 0.555087 | 0.918554 |  -1.67696  | 0.604306 | 0.256754   | 0.243577  |       0.364459  |   0.480057  |       5 |     10.9912  |
| TXNIP  | professional antigen presenting cell                    |      1.97112 |        15023 |       0.158576    |          0.0184019   |              64385 | 1.02815     | 0.555087 | 0.918554 |  -1.67696  | 0.604306 | 0.256754   | 0.243577  |       0.364459  |   0.480057  |       5 |     10.9912  |
| ATP1B3 | stem cell of epidermis                                  |      2.20196 |        25431 |       0.784593    |          0.0311509   |              47452 | 2.41508     | 1.11238  | 1.05872  |  -3.10141  | 1.05068  | 0.66408    | 0.815663  |       0.227667  |   0.589378  |       5 |     10.8365  |
| MAF    | stem cell of epidermis                                  |      1.79559 |          907 |       0.0279826   |          0.001111    |              47452 | 0.32507     | 1.11238  | 1.05872  |  -3.10141  | 1.05068  | 0.66408    | 0.815663  |       0.227667  |   0.589378  |       5 |     10.8365  |
| MMP13  | stem cell of epidermis                                  |      1.70608 |          151 |       0.00465862  |          0.000184962 |              47452 | 0.204505    | 1.11238  | 1.05872  |  -3.10141  | 1.05068  | 0.66408    | 0.815663  |       0.227667  |   0.589378  |       5 |     10.8365  |
| PLAU   | stem cell of epidermis                                  |      1.70676 |          381 |       0.0117545   |          0.000466693 |              47452 | 0.221827    | 1.11238  | 1.05872  |  -3.10141  | 1.05068  | 0.66408    | 0.815663  |       0.227667  |   0.589378  |       5 |     10.8365  |
| TXNIP  | stem cell of epidermis                                  |      2.24417 |        20582 |       0.65894     |          0.0252112   |              47452 | 2.12713     | 1.11238  | 1.05872  |  -3.10141  | 1.05068  | 0.66408    | 0.815663  |       0.227667  |   0.589378  |       5 |     10.8365  |
| ATP1B3 | macrophage                                              |      1.95562 |        14190 |       0.280634    |          0.0173816   |              45686 | 1.17756     | 0.51098  | 0.892632 |  -1.42783  | 0.572442 | 0.224516   | 0.20274   |       0.388383  |   0.45984   |       5 |     10.8065  |
| MAF    | macrophage                                              |      2.17852 |        16637 |       0.329029    |          0.0203789   |              45686 | 1.47841     | 0.51098  | 0.892632 |  -1.42783  | 0.572442 | 0.224516   | 0.20274   |       0.388383  |   0.45984   |       5 |     10.8065  |
| MMP13  | macrophage                                              |      1.7172  |          142 |       0.00289571  |          0.000173938 |              45686 | 0.209728    | 0.51098  | 0.892632 |  -1.42783  | 0.572442 | 0.224516   | 0.20274   |       0.388383  |   0.45984   |       5 |     10.8065  |
| PLAU   | macrophage                                              |      1.92977 |         3207 |       0.0634384   |          0.00392831  |              45686 | 0.537876    | 0.51098  | 0.892632 |  -1.42783  | 0.572442 | 0.224516   | 0.20274   |       0.388383  |   0.45984   |       5 |     10.8065  |
| TXNIP  | macrophage                                              |      2.0016  |        11510 |       0.227632    |          0.0140988   |              45686 | 1.05959     | 0.51098  | 0.892632 |  -1.42783  | 0.572442 | 0.224516   | 0.20274   |       0.388383  |   0.45984   |       5 |     10.8065  |
| ATP1B3 | basal cell of epidermis                                 |      2.20972 |        24813 |       0.794397    |          0.0303939   |              46715 | 2.41885     | 1.1185   | 1.05434  |  -3.09513  | 1.06085  | 0.668554   | 0.828695  |       0.228269  |   0.585963  |       5 |     10.7525  |
| MAF    | basal cell of epidermis                                 |      1.80535 |          834 |       0.0267008   |          0.00102158  |              46715 | 0.328302    | 1.1185   | 1.05434  |  -3.09513  | 1.06085  | 0.668554   | 0.828695  |       0.228269  |   0.585963  |       5 |     10.7525  |
| MMP13  | basal cell of epidermis                                 |      1.68844 |          149 |       0.00477029  |          0.000182513 |              46715 | 0.191759    | 1.1185   | 1.05434  |  -3.09513  | 1.06085  | 0.668554   | 0.828695  |       0.228269  |   0.585963  |       5 |     10.7525  |
| PLAU   | basal cell of epidermis                                 |      1.68823 |          337 |       0.0107892   |          0.000412797 |              46715 | 0.205683    | 1.1185   | 1.05434  |  -3.09513  | 1.06085  | 0.668554   | 0.828695  |       0.228269  |   0.585963  |       5 |     10.7525  |
| TXNIP  | basal cell of epidermis                                 |      2.24417 |        20582 |       0.65894     |          0.0252112   |              46715 | 2.12713     | 1.1185   | 1.05434  |  -3.09513  | 1.06085  | 0.668554   | 0.828695  |       0.228269  |   0.585963  |       5 |     10.7525  |
| ATP1B3 | mature T cell                                           |      1.99876 |        31622 |       0.214816    |          0.0387343   |              91929 | 1.57857     | 0.939014 | 1.02664  |  -1.18557  | 0.91465  | 0.537367   | 0.641324  |       0.411648  |   0.564354  |       5 |     10.7203  |
| MAF    | mature T cell                                           |      1.86831 |        13511 |       0.0917767   |          0.0165499   |              91929 | 0.812366    | 0.939014 | 1.02664  |  -1.18557  | 0.91465  | 0.537367   | 0.641324  |       0.411648  |   0.564354  |       5 |     10.7203  |
| MMP13  | mature T cell                                           |      1.67873 |          986 |       0.00923982  |          0.00120777  |              91929 | 0.213866    | 0.939014 | 1.02664  |  -1.18557  | 0.91465  | 0.537367   | 0.641324  |       0.411648  |   0.564354  |       5 |     10.7203  |
| PLAU   | mature T cell                                           |      1.65934 |          227 |       0.00172392  |          0.000278056 |              91929 | 0.168065    | 0.939014 | 1.02664  |  -1.18557  | 0.91465  | 0.537367   | 0.641324  |       0.411648  |   0.564354  |       5 |     10.7203  |
| TXNIP  | mature T cell                                           |      2.36148 |        45583 |       0.309633    |          0.0558354   |              91929 | 2.36032     | 0.939014 | 1.02664  |  -1.18557  | 0.91465  | 0.537367   | 0.641324  |       0.411648  |   0.564354  |       5 |     10.7203  |
| ATP1B3 | connective tissue cell                                  |      1.82118 |        30844 |       0.118108    |          0.0377813   |             114128 | 1.28313     | 0.918472 | 1.0574   |   1.68368  | 0.86861  | 0.522354   | 0.582318  |       0.687187  |   0.58835   |       5 |     10.6532  |
| MAF    | connective tissue cell                                  |      1.79169 |        14238 |       0.0545202   |          0.0174404   |             114128 | 0.720145    | 0.918472 | 1.0574   |   1.68368  | 0.86861  | 0.522354   | 0.582318  |       0.687187  |   0.58835   |       5 |     10.6532  |
| MMP13  | connective tissue cell                                  |      1.72222 |         1135 |       0.00446443  |          0.00139028  |             114128 | 0.24242     | 0.918472 | 1.0574   |   1.68368  | 0.86861  | 0.522354   | 0.582318  |       0.687187  |   0.58835   |       5 |     10.6532  |
| PLAU   | connective tissue cell                                  |      1.93535 |         4029 |       0.0154295   |          0.00493519  |             114128 | 0.492014    | 0.918472 | 1.0574   |   1.68368  | 0.86861  | 0.522354   | 0.582318  |       0.687187  |   0.58835   |       5 |     10.6532  |
| TXNIP  | connective tissue cell                                  |      2.08026 |        63882 |       0.244617    |          0.0782501   |             114128 | 2.54932     | 0.918472 | 1.0574   |   1.68368  | 0.86861  | 0.522354   | 0.582318  |       0.687187  |   0.58835   |       5 |     10.6532  |
| ATP1B3 | progenitor cell                                         |      2.18338 |        26390 |       0.636164    |          0.0323256   |              51021 | 2.20466     | 1.00773  | 0.999214 |  -3.16391  | 1.00853  | 0.587596   | 0.761641  |       0.221664  |   0.542966  |       5 |     10.4597  |
| MAF    | progenitor cell                                         |      1.8011  |         1407 |       0.0344304   |          0.00172346  |              51021 | 0.352191    | 1.00773  | 0.999214 |  -3.16391  | 1.00853  | 0.587596   | 0.761641  |       0.221664  |   0.542966  |       5 |     10.4597  |
| MMP13  | progenitor cell                                         |      1.68209 |          172 |       0.00428831  |          0.000210686 |              51021 | 0.187025    | 1.00773  | 0.999214 |  -3.16391  | 1.00853  | 0.587596   | 0.761641  |       0.221664  |   0.542966  |       5 |     10.4597  |
| PLAU   | progenitor cell                                         |      1.75005 |          551 |       0.0134456   |          0.000674929 |              51021 | 0.260496    | 1.00773  | 0.999214 |  -3.16391  | 1.00853  | 0.587596   | 0.761641  |       0.221664  |   0.542966  |       5 |     10.4597  |
| TXNIP  | progenitor cell                                         |      2.22683 |        22501 |       0.542716    |          0.0275619   |              51021 | 1.9917      | 1.00773  | 0.999214 |  -3.16391  | 1.00853  | 0.587596   | 0.761641  |       0.221664  |   0.542966  |       5 |     10.4597  |
| ATP1B3 | stem cell                                               |      2.20121 |        25507 |       0.696095    |          0.031244    |              47874 | 2.28384     | 1.04886  | 1.01054  |  -3.1164   | 1.03792  | 0.617652   | 0.79931   |       0.226227  |   0.551797  |       5 |     10.4388  |
| MAF    | stem cell                                               |      1.80317 |          929 |       0.0253832   |          0.00113795  |              47874 | 0.327283    | 1.04886  | 1.01054  |  -3.1164   | 1.03792  | 0.617652   | 0.79931   |       0.226227  |   0.551797  |       5 |     10.4388  |
| MMP13  | stem cell                                               |      1.68779 |          190 |       0.00523243  |          0.000232734 |              47874 | 0.193075    | 1.04886  | 1.01054  |  -3.1164   | 1.03792  | 0.617652   | 0.79931   |       0.226227  |   0.551797  |       5 |     10.4388  |
| PLAU   | stem cell                                               |      1.71234 |          394 |       0.0107653   |          0.000482617 |              47874 | 0.224756    | 1.04886  | 1.01054  |  -3.1164   | 1.03792  | 0.617652   | 0.79931   |       0.226227  |   0.551797  |       5 |     10.4388  |
| TXNIP  | stem cell                                               |      2.23843 |        20854 |       0.587933    |          0.0255444   |              47874 | 2.02373     | 1.04886  | 1.01054  |  -3.1164   | 1.03792  | 0.617652   | 0.79931   |       0.226227  |   0.551797  |       5 |     10.4388  |
| ATP1B3 | CL:0000723                                              |      2.20096 |        25489 |       0.701095    |          0.0312219   |              47813 | 2.29067     | 1.05429  | 1.01037  |  -3.1168   | 1.04347  | 0.62162    | 0.806418  |       0.226189  |   0.551666  |       5 |     10.4149  |
| MAF    | CL:0000723                                              |      1.79368 |          918 |       0.0252809   |          0.00112447  |              47813 | 0.319915    | 1.05429  | 1.01037  |  -3.1168   | 1.04347  | 0.62162    | 0.806418  |       0.226189  |   0.551666  |       5 |     10.4149  |
| MMP13  | CL:0000723                                              |      1.68779 |          190 |       0.00523243  |          0.000232734 |              47813 | 0.193075    | 1.05429  | 1.01037  |  -3.1168   | 1.04347  | 0.62162    | 0.806418  |       0.226189  |   0.551666  |       5 |     10.4149  |
| PLAU   | CL:0000723                                              |      1.70532 |          391 |       0.0107678   |          0.000478942 |              47813 | 0.219566    | 1.05429  | 1.01037  |  -3.1168   | 1.04347  | 0.62162    | 0.806418  |       0.226189  |   0.551666  |       5 |     10.4149  |
| TXNIP  | CL:0000723                                              |      2.23799 |        20825 |       0.591922    |          0.0255089   |              47813 | 2.02862     | 1.05429  | 1.01037  |  -3.1168   | 1.04347  | 0.62162    | 0.806418  |       0.226189  |   0.551666  |       5 |     10.4149  |
| ATP1B3 | epithelial fate stem cell                               |      2.20872 |        24869 |       0.707833    |          0.0304625   |              47068 | 2.28978     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| MAF    | epithelial fate stem cell                               |      1.80314 |          845 |       0.0240508   |          0.00103505  |              47068 | 0.323012    | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| MMP13  | epithelial fate stem cell                               |      1.67361 |          188 |       0.00535094  |          0.000230284 |              47068 | 0.182865    | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| PLAU   | epithelial fate stem cell                               |      1.68714 |          347 |       0.00987647  |          0.000425046 |              47068 | 0.20379     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| TXNIP  | epithelial fate stem cell                               |      2.23814 |        20819 |       0.59256     |          0.0255015   |              47068 | 2.02952     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| ATP1B3 | basal cell                                              |      2.20872 |        24869 |       0.707833    |          0.0304625   |              47068 | 2.28978     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| MAF    | basal cell                                              |      1.80314 |          845 |       0.0240508   |          0.00103505  |              47068 | 0.323012    | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| MMP13  | basal cell                                              |      1.67361 |          188 |       0.00535094  |          0.000230284 |              47068 | 0.182865    | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| PLAU   | basal cell                                              |      1.68714 |          347 |       0.00987647  |          0.000425046 |              47068 | 0.20379     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| TXNIP  | basal cell                                              |      2.23814 |        20819 |       0.59256     |          0.0255015   |              47068 | 2.02952     | 1.05869  | 1.00579  |  -3.11833  | 1.05259  | 0.624836   | 0.818108  |       0.226041  |   0.548098  |       5 |     10.3354  |
| ATP1B3 | fibroblast                                              |      1.80838 |        28138 |       0.120089    |          0.0344667   |             105848 | 1.20407     | 0.869236 | 1.0215   |   1.90981  | 0.850941 | 0.486367   | 0.559672  |       0.708902  |   0.560347  |       5 |     10.335   |
| MAF    | fibroblast                                              |      1.79198 |        13997 |       0.0597374   |          0.0171452   |             105848 | 0.721708    | 0.869236 | 1.0215   |   1.90981  | 0.850941 | 0.486367   | 0.559672  |       0.708902  |   0.560347  |       5 |     10.335   |
| MMP13  | fibroblast                                              |      1.75692 |          686 |       0.00292987  |          0.000840293 |             105848 | 0.253353    | 0.869236 | 1.0215   |   1.90981  | 0.850941 | 0.486367   | 0.559672  |       0.708902  |   0.560347  |       5 |     10.335   |
| PLAU   | fibroblast                                              |      1.93656 |         3716 |       0.0158594   |          0.00455179  |             105848 | 0.485132    | 0.869236 | 1.0215   |   1.90981  | 0.850941 | 0.486367   | 0.559672  |       0.708902  |   0.560347  |       5 |     10.335   |
| TXNIP  | fibroblast                                              |      2.08567 |        59311 |       0.253132    |          0.072651    |             105848 | 2.44324     | 0.869236 | 1.0215   |   1.90981  | 0.850941 | 0.486367   | 0.559672  |       0.708902  |   0.560347  |       5 |     10.335   |
| ATP1B3 | skin fibroblast                                         |      1.83878 |        19174 |       0.169566    |          0.0234866   |              69807 | 1.05965     | 0.691157 | 0.977011 |   2.75644  | 0.70742  | 0.356208   | 0.375732  |       0.790206  |   0.525649  |       5 |     10.3148  |
| MAF    | skin fibroblast                                         |      1.84161 |         9418 |       0.0832884   |          0.0115363   |              69807 | 0.670217    | 0.691157 | 0.977011 |   2.75644  | 0.70742  | 0.356208   | 0.375732  |       0.790206  |   0.525649  |       5 |     10.3148  |
| MMP13  | skin fibroblast                                         |      2.22251 |            1 |       8.84353e-06 |          1.22492e-06 |              69807 | 0.570055    | 0.691157 | 0.977011 |   2.75644  | 0.70742  | 0.356208   | 0.375732  |       0.790206  |   0.525649  |       5 |     10.3148  |
| PLAU   | skin fibroblast                                         |      1.93228 |         2097 |       0.0185449   |          0.00256865  |              69807 | 0.442546    | 0.691157 | 0.977011 |   2.75644  | 0.70742  | 0.356208   | 0.375732  |       0.790206  |   0.525649  |       5 |     10.3148  |
| TXNIP  | skin fibroblast                                         |      2.22643 |        39117 |       0.345932    |          0.0479151   |              69807 | 2.14259     | 0.691157 | 0.977011 |   2.75644  | 0.70742  | 0.356208   | 0.375732  |       0.790206  |   0.525649  |       5 |     10.3148  |
| ATP1B3 | hair follicular keratinocyte                            |      2.2172  |         2852 |       0.839812    |          0.00349346  |               5207 | 1.90247     | 0.927636 | 0.956212 |  -5.47215  | 0.970115 | 0.529051   | 0.71241   |       0         |   0.509428  |       4 |     10.1073  |
| MAF    | hair follicular keratinocyte                            |      1.61235 |           27 |       0.00795053  |          3.30728e-05 |               5207 | 0.137768    | 0.927636 | 0.956212 |  -5.47215  | 0.970115 | 0.529051   | 0.71241   |       0         |   0.509428  |       4 |     10.1073  |
| PLAU   | hair follicular keratinocyte                            |      1.68247 |           13 |       0.00382803  |          1.59239e-05 |               5207 | 0.18234     | 0.927636 | 0.956212 |  -5.47215  | 0.970115 | 0.529051   | 0.71241   |       0         |   0.509428  |       4 |     10.1073  |
| TXNIP  | hair follicular keratinocyte                            |      2.15056 |         2315 |       0.681684    |          0.00283568  |               5207 | 1.60227     | 0.927636 | 0.956212 |  -5.47215  | 0.970115 | 0.529051   | 0.71241   |       0         |   0.509428  |       4 |     10.1073  |
| ATP1B3 | precursor cell                                          |      2.16921 |        27417 |       0.521087    |          0.0335835   |              53753 | 2.0493      | 0.934877 | 0.940823 |  -3.19395  | 0.99368  | 0.534344   | 0.742611  |       0.21878   |   0.497426  |       5 |      9.97249 |
| MAF    | precursor cell                                          |      1.77136 |         1783 |       0.0342904   |          0.00218403  |              53753 | 0.340397    | 0.934877 | 0.940823 |  -3.19395  | 0.99368  | 0.534344   | 0.742611  |       0.21878   |   0.497426  |       5 |      9.97249 |
| MMP13  | precursor cell                                          |      1.68273 |          213 |       0.00471385  |          0.000260907 |              53753 | 0.189229    | 0.934877 | 0.940823 |  -3.19395  | 0.99368  | 0.534344   | 0.742611  |       0.21878   |   0.497426  |       5 |      9.97249 |
| PLAU   | precursor cell                                          |      1.71888 |          895 |       0.0171745   |          0.0010963   |              53753 | 0.252596    | 0.934877 | 0.940823 |  -3.19395  | 0.99368  | 0.534344   | 0.742611  |       0.21878   |   0.497426  |       5 |      9.97249 |
| TXNIP  | precursor cell                                          |      2.20711 |        23445 |       0.455995    |          0.0287182   |              53753 | 1.87259     | 0.934877 | 0.940823 |  -3.19395  | 0.99368  | 0.534344   | 0.742611  |       0.21878   |   0.497426  |       5 |      9.97249 |
| ATP1B3 | mast cell                                               |      2.11348 |         1206 |       0.277624    |          0.00147725  |               3116 | 0.939353    | 0.348643 | 0.698903 |  -1.93209  | 0.498843 | 0.105863   | 0.108413  |       0.339958  |   0.308747  |       5 |      9.21399 |
| MAF    | mast cell                                               |      2.14928 |          389 |       0.0895488   |          0.000476493 |               3116 | 0.661395    | 0.348643 | 0.698903 |  -1.93209  | 0.498843 | 0.105863   | 0.108413  |       0.339958  |   0.308747  |       5 |      9.21399 |
| MMP13  | mast cell                                               |      1.87773 |           35 |       0.00805709  |          4.28721e-05 |               3116 | 0.331645    | 0.348643 | 0.698903 |  -1.93209  | 0.498843 | 0.105863   | 0.108413  |       0.339958  |   0.308747  |       5 |      9.21399 |
| PLAU   | mast cell                                               |      1.92606 |          145 |       0.0333794   |          0.000177613 |               3116 | 0.407827    | 0.348643 | 0.698903 |  -1.93209  | 0.498843 | 0.105863   | 0.108413  |       0.339958  |   0.308747  |       5 |      9.21399 |
| TXNIP  | mast cell                                               |      2.33936 |         1341 |       0.308702    |          0.00164261  |               3116 | 1.15429     | 0.348643 | 0.698903 |  -1.93209  | 0.498843 | 0.105863   | 0.108413  |       0.339958  |   0.308747  |       5 |      9.21399 |
| ATP1B3 | endothelial cell                                        |      2.07187 |        23014 |       0.525206    |          0.0281902   |              38284 | 1.86622     | 0.713042 | 0.826887 |  -1.10956  | 0.862321 | 0.372204   | 0.574258  |       0.418947  |   0.408564  |       5 |      8.98908 |
| MAF    | endothelial cell                                        |      1.8268  |          736 |       0.0167964   |          0.000901539 |              38284 | 0.326454    | 0.713042 | 0.826887 |  -1.10956  | 0.862321 | 0.372204   | 0.574258  |       0.418947  |   0.408564  |       5 |      8.98908 |
| MMP13  | endothelial cell                                        |      1.71662 |          226 |       0.00587272  |          0.000276831 |              38284 | 0.216024    | 0.713042 | 0.826887 |  -1.10956  | 0.862321 | 0.372204   | 0.574258  |       0.418947  |   0.408564  |       5 |      8.98908 |
| PLAU   | endothelial cell                                        |      1.96624 |         1230 |       0.0280771   |          0.00150665  |              38284 | 0.458315    | 0.713042 | 0.826887 |  -1.10956  | 0.862321 | 0.372204   | 0.574258  |       0.418947  |   0.408564  |       5 |      8.98908 |
| TXNIP  | endothelial cell                                        |      2.08317 |        13078 |       0.298455    |          0.0160195   |              38284 | 1.26742     | 0.713042 | 0.826887 |  -1.10956  | 0.862321 | 0.372204   | 0.574258  |       0.418947  |   0.408564  |       5 |      8.98908 |
| ATP1B3 | naive thymus-derived CD8-positive, alpha-beta T cell    |      2.03345 |         1302 |       0.29457     |          0.00159484  |               2651 | 0.908996    | 0.279754 | 0.675321 | nan        | 0.414253 | 0.0555116  | 0         |     nan         |   0.290355  |       3 |      8.82768 |
| MAF    | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.84033 |          202 |       0.0457014   |          0.000247433 |               2651 | 0.365328    | 0.279754 | 0.675321 | nan        | 0.414253 | 0.0555116  | 0         |     nan         |   0.290355  |       3 |      8.82768 |
| TXNIP  | naive thymus-derived CD8-positive, alpha-beta T cell    |      1.89549 |         1147 |       0.259502    |          0.00140498  |               2651 | 0.751637    | 0.279754 | 0.675321 | nan        | 0.414253 | 0.0555116  | 0         |     nan         |   0.290355  |       3 |      8.82768 |
| ATP1B3 | alpha-beta T cell                                       |      2.05807 |        20506 |       0.312144    |          0.0251181   |              44270 | 1.46919     | 0.599986 | 0.751196 |  -2.71185  | 0.798708 | 0.289571   | 0.492729  |       0.265077  |   0.349531  |       5 |      8.79869 |
| MAF    | alpha-beta T cell                                       |      1.87156 |         6952 |       0.105806    |          0.00851562  |              44270 | 0.659587    | 0.599986 | 0.751196 |  -2.71185  | 0.798708 | 0.289571   | 0.492729  |       0.265077  |   0.349531  |       5 |      8.79869 |
| MMP13  | alpha-beta T cell                                       |      1.67738 |          218 |       0.00789455  |          0.000267032 |              44270 | 0.190236    | 0.599986 | 0.751196 |  -2.71185  | 0.798708 | 0.289571   | 0.492729  |       0.265077  |   0.349531  |       5 |      8.79869 |
| PLAU   | alpha-beta T cell                                       |      1.6696  |           75 |       0.00149745  |          9.18688e-05 |              44270 | 0.171122    | 0.599986 | 0.751196 |  -2.71185  | 0.798708 | 0.289571   | 0.492729  |       0.265077  |   0.349531  |       5 |      8.79869 |
| TXNIP  | alpha-beta T cell                                       |      2.05101 |        16519 |       0.251412    |          0.0202344   |              44270 | 1.26584     | 0.599986 | 0.751196 |  -2.71185  | 0.798708 | 0.289571   | 0.492729  |       0.265077  |   0.349531  |       5 |      8.79869 |
| ATP1B3 | mature alpha-beta T cell                                |      2.05816 |        20502 |       0.312469    |          0.0251132   |              44227 | 1.46964     | 0.599826 | 0.751053 |  -2.70609  | 0.798646 | 0.289454   | 0.49265   |       0.265629  |   0.34942   |       5 |      8.79641 |
| MAF    | mature alpha-beta T cell                                |      1.87161 |         6951 |       0.105922    |          0.0085144   |              44227 | 0.659767    | 0.599826 | 0.751053 |  -2.70609  | 0.798646 | 0.289454   | 0.49265   |       0.265629  |   0.34942   |       5 |      8.79641 |
| MMP13  | mature alpha-beta T cell                                |      1.67738 |          218 |       0.00789455  |          0.000267032 |              44227 | 0.190236    | 0.599826 | 0.751053 |  -2.70609  | 0.798646 | 0.289454   | 0.49265   |       0.265629  |   0.34942   |       5 |      8.79641 |
| PLAU   | mature alpha-beta T cell                                |      1.6696  |           75 |       0.00149745  |          9.18688e-05 |              44227 | 0.171122    | 0.599826 | 0.751053 |  -2.70609  | 0.798646 | 0.289454   | 0.49265   |       0.265629  |   0.34942   |       5 |      8.79641 |
| TXNIP  | mature alpha-beta T cell                                |      2.05112 |        16481 |       0.251143    |          0.0201879   |              44227 | 1.2645      | 0.599826 | 0.751053 |  -2.70609  | 0.798646 | 0.289454   | 0.49265   |       0.265629  |   0.34942   |       5 |      8.79641 |
| ATP1B3 | endothelial cell of vascular tree                       |      2.0668  |        20659 |       0.536681    |          0.0253056   |              33675 | 1.81647     | 0.701841 | 0.790041 |  -1.12984  | 0.88836  | 0.364017   | 0.607631  |       0.417     |   0.379828  |       5 |      8.54029 |
| MAF    | endothelial cell of vascular tree                       |      1.86569 |          566 |       0.0147036   |          0.000693303 |              33675 | 0.347098    | 0.701841 | 0.790041 |  -1.12984  | 0.88836  | 0.364017   | 0.607631  |       0.417     |   0.379828  |       5 |      8.54029 |
| MMP13  | endothelial cell of vascular tree                       |      1.71662 |          226 |       0.00587272  |          0.000276831 |              33675 | 0.216024    | 0.701841 | 0.790041 |  -1.12984  | 0.88836  | 0.364017   | 0.607631  |       0.417     |   0.379828  |       5 |      8.54029 |
| PLAU   | endothelial cell of vascular tree                       |      1.84174 |          771 |       0.0200348   |          0.000944411 |              33675 | 0.343138    | 0.701841 | 0.790041 |  -1.12984  | 0.88836  | 0.364017   | 0.607631  |       0.417     |   0.379828  |       5 |      8.54029 |
| TXNIP  | endothelial cell of vascular tree                       |      2.09016 |        11453 |       0.297527    |          0.014029    |              33675 | 1.22747     | 0.701841 | 0.790041 |  -1.12984  | 0.88836  | 0.364017   | 0.607631  |       0.417     |   0.379828  |       5 |      8.54029 |
| ATP1B3 | prickle cell                                            |      1.79982 |        15325 |       0.582235    |          0.0187718   |              27607 | 1.54685     | 0.734668 | 0.773438 |  -5.17137  | 0.949874 | 0.388011   | 0.686468  |       0.0288848 |   0.366878  |       4 |      8.47525 |
| MAF    | prickle cell                                            |      1.58369 |          964 |       0.0366247   |          0.00118082  |              27607 | 0.185052    | 0.734668 | 0.773438 |  -5.17137  | 0.949874 | 0.388011   | 0.686468  |       0.0288848 |   0.366878  |       4 |      8.47525 |
| PLAU   | prickle cell                                            |      1.57571 |          108 |       0.00410319  |          0.000132291 |              27607 | 0.107456    | 0.734668 | 0.773438 |  -5.17137  | 0.949874 | 0.388011   | 0.686468  |       0.0288848 |   0.366878  |       4 |      8.47525 |
| TXNIP  | prickle cell                                            |      1.87196 |        11210 |       0.425896    |          0.0137313   |              27607 | 1.2544      | 0.734668 | 0.773438 |  -5.17137  | 0.949874 | 0.388011   | 0.686468  |       0.0288848 |   0.366878  |       4 |      8.47525 |
| ATP1B3 | secretory cell                                          |      2.11102 |         1266 |       0.189777    |          0.00155074  |               3267 | 0.807394    | 0.295366 | 0.61792  |  -1.40253  | 0.478    | 0.0669229  | 0.0817007 |       0.390813  |   0.245587  |       5 |      8.35934 |
| MAF    | secretory cell                                          |      2.14297 |          395 |       0.0604345   |          0.000483842 |               3267 | 0.613284    | 0.295366 | 0.61792  |  -1.40253  | 0.478    | 0.0669229  | 0.0817007 |       0.390813  |   0.245587  |       5 |      8.35934 |
| MMP13  | secretory cell                                          |      1.76192 |           88 |       0.0134639   |          0.000107793 |               3267 | 0.256739    | 0.295366 | 0.61792  |  -1.40253  | 0.478    | 0.0669229  | 0.0817007 |       0.390813  |   0.245587  |       5 |      8.35934 |
| PLAU   | secretory cell                                          |      1.95996 |          153 |       0.0229351   |          0.000187412 |               3267 | 0.417091    | 0.295366 | 0.61792  |  -1.40253  | 0.478    | 0.0669229  | 0.0817007 |       0.390813  |   0.245587  |       5 |      8.35934 |
| TXNIP  | secretory cell                                          |      2.33432 |         1365 |       0.204586    |          0.00167201  |               3267 | 0.995092    | 0.295366 | 0.61792  |  -1.40253  | 0.478    | 0.0669229  | 0.0817007 |       0.390813  |   0.245587  |       5 |      8.35934 |
| ATP1B3 | endothelial cell of lymphatic vessel                    |      1.98413 |         2926 |       0.465331    |          0.00358411  |               5501 | 1.1728      | 0.440731 | 0.641348 |  -2.72955  | 0.687195 | 0.17317    | 0.34981   |       0.263377  |   0.263858  |       5 |      8.1213  |
| MAF    | endothelial cell of lymphatic vessel                    |      1.89042 |          340 |       0.0540712   |          0.000416472 |               5501 | 0.418111    | 0.440731 | 0.641348 |  -2.72955  | 0.687195 | 0.17317    | 0.34981   |       0.263377  |   0.263858  |       5 |      8.1213  |
| MMP13  | endothelial cell of lymphatic vessel                    |      1.67265 |           50 |       0.00795165  |          6.12458e-05 |               5501 | 0.182354    | 0.440731 | 0.641348 |  -2.72955  | 0.687195 | 0.17317    | 0.34981   |       0.263377  |   0.263858  |       5 |      8.1213  |
| PLAU   | endothelial cell of lymphatic vessel                    |      1.96358 |            7 |       0.00111323  |          8.57442e-06 |               5501 | 0.383079    | 0.440731 | 0.641348 |  -2.72955  | 0.687195 | 0.17317    | 0.34981   |       0.263377  |   0.263858  |       5 |      8.1213  |
| TXNIP  | endothelial cell of lymphatic vessel                    |      2.08853 |         2178 |       0.346374    |          0.00266787  |               5501 | 1.0504      | 0.440731 | 0.641348 |  -2.72955  | 0.687195 | 0.17317    | 0.34981   |       0.263377  |   0.263858  |       5 |      8.1213  |
| ATP1B3 | regulatory T cell                                       |      2.07479 |         8255 |       0.245502    |          0.0101117   |              21592 | 1.05232     | 0.43267  | 0.620733 |  -2.97137  | 0.69703  | 0.167279   | 0.362415  |       0.240155  |   0.247781  |       5 |      7.92924 |
| MAF    | regulatory T cell                                       |      1.88688 |         4552 |       0.135375    |          0.00557582  |              21592 | 0.650635    | 0.43267  | 0.620733 |  -2.97137  | 0.69703  | 0.167279   | 0.362415  |       0.240155  |   0.247781  |       5 |      7.92924 |
| MMP13  | regulatory T cell                                       |      1.68809 |          288 |       0.011405    |          0.000352776 |              21592 | 0.205188    | 0.43267  | 0.620733 |  -2.97137  | 0.69703  | 0.167279   | 0.362415  |       0.240155  |   0.247781  |       5 |      7.92924 |
| PLAU   | regulatory T cell                                       |      1.65088 |           58 |       0.00172491  |          7.10452e-05 |              21592 | 0.157355    | 0.43267  | 0.620733 |  -2.97137  | 0.69703  | 0.167279   | 0.362415  |       0.240155  |   0.247781  |       5 |      7.92924 |
| TXNIP  | regulatory T cell                                       |      2.03735 |         8439 |       0.250974    |          0.0103371   |              21592 | 1.03817     | 0.43267  | 0.620733 |  -2.97137  | 0.69703  | 0.167279   | 0.362415  |       0.240155  |   0.247781  |       5 |      7.92924 |
| ATP1B3 | CD8-positive, alpha-beta T cell                         |      2.07732 |        16216 |       0.334523    |          0.0198633   |              31371 | 1.40155     | 0.551626 | 0.674045 |  -2.09369  | 0.818381 | 0.254224   | 0.517943  |       0.324439  |   0.28936   |       5 |      7.85783 |
| MAF    | CD8-positive, alpha-beta T cell                         |      1.87958 |         3985 |       0.0822073   |          0.00488129  |              31371 | 0.550332    | 0.551626 | 0.674045 |  -2.09369  | 0.818381 | 0.254224   | 0.517943  |       0.324439  |   0.28936   |       5 |      7.85783 |
| MMP13  | CD8-positive, alpha-beta T cell                         |      1.67738 |          218 |       0.00789455  |          0.000267032 |              31371 | 0.190236    | 0.551626 | 0.674045 |  -2.09369  | 0.818381 | 0.254224   | 0.517943  |       0.324439  |   0.28936   |       5 |      7.85783 |
| PLAU   | CD8-positive, alpha-beta T cell                         |      1.64116 |           57 |       0.00133543  |          6.98203e-05 |              31371 | 0.149659    | 0.551626 | 0.674045 |  -2.09369  | 0.818381 | 0.254224   | 0.517943  |       0.324439  |   0.28936   |       5 |      7.85783 |
| TXNIP  | CD8-positive, alpha-beta T cell                         |      2.05604 |        10895 |       0.224755    |          0.0133455   |              31371 | 1.07845     | 0.551626 | 0.674045 |  -2.09369  | 0.818381 | 0.254224   | 0.517943  |       0.324439  |   0.28936   |       5 |      7.85783 |
| ATP1B3 | neoplastic cell                                         |      1.87792 |         5747 |       0.220665    |          0.0070396   |              10599 | 0.804143    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MAF    | neoplastic cell                                         |      1.77166 |          392 |       0.0150515   |          0.000480167 |              10599 | 0.27439     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MMP13  | neoplastic cell                                         |      1.74236 |           18 |       0.000691138 |          2.20485e-05 |              10599 | 0.221439    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| PLAU   | neoplastic cell                                         |      2.30923 |         1084 |       0.0416219   |          0.00132781  |              10599 | 0.724799    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| TXNIP  | neoplastic cell                                         |      1.89133 |         3358 |       0.128936    |          0.00411327  |              10599 | 0.61215     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| ATP1B3 | malignant cell                                          |      1.87792 |         5747 |       0.220665    |          0.0070396   |              10599 | 0.804143    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MAF    | malignant cell                                          |      1.77166 |          392 |       0.0150515   |          0.000480167 |              10599 | 0.27439     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MMP13  | malignant cell                                          |      1.74236 |           18 |       0.000691138 |          2.20485e-05 |              10599 | 0.221439    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| PLAU   | malignant cell                                          |      2.30923 |         1084 |       0.0416219   |          0.00132781  |              10599 | 0.724799    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| TXNIP  | malignant cell                                          |      1.89133 |         3358 |       0.128936    |          0.00411327  |              10599 | 0.61215     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| ATP1B3 | abnormal cell                                           |      1.87792 |         5747 |       0.220665    |          0.0070396   |              10599 | 0.804143    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MAF    | abnormal cell                                           |      1.77166 |          392 |       0.0150515   |          0.000480167 |              10599 | 0.27439     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| MMP13  | abnormal cell                                           |      1.74236 |           18 |       0.000691138 |          2.20485e-05 |              10599 | 0.221439    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| PLAU   | abnormal cell                                           |      2.30923 |         1084 |       0.0416219   |          0.00132781  |              10599 | 0.724799    | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| TXNIP  | abnormal cell                                           |      1.89133 |         3358 |       0.128936    |          0.00411327  |              10599 | 0.61215     | 0.264746 | 0.527384 |  -2.83592  | 0.501999 | 0.0445426  | 0.112458  |       0.253162  |   0.174976  |       5 |      7.62934 |
| ATP1B3 | neural cell                                             |      2.6321  |         1763 |       0.215842    |          0.00215953  |               4497 | 1.23979     | 0.389289 | 0.629092 |   1.20602  | 0.61881  | 0.135571   | 0.262167  |       0.641316  |   0.2543    |       5 |      7.48209 |
| MAF    | neural cell                                             |      1.94996 |          683 |       0.0971827   |          0.000836618 |               4497 | 0.535406    | 0.389289 | 0.629092 |   1.20602  | 0.61881  | 0.135571   | 0.262167  |       0.641316  |   0.2543    |       5 |      7.48209 |
| MMP13  | neural cell                                             |      1.7049  |           35 |       0.0185087   |          4.28721e-05 |               4497 | 0.221304    | 0.389289 | 0.629092 |   1.20602  | 0.61881  | 0.135571   | 0.262167  |       0.641316  |   0.2543    |       5 |      7.48209 |
| PLAU   | neural cell                                             |      1.91888 |          284 |       0.0377911   |          0.000347876 |               4497 | 0.412937    | 0.389289 | 0.629092 |   1.20602  | 0.61881  | 0.135571   | 0.262167  |       0.641316  |   0.2543    |       5 |      7.48209 |
| TXNIP  | neural cell                                             |      1.95015 |         1732 |       0.212047    |          0.00212156  |               4497 | 0.73602     | 0.389289 | 0.629092 |   1.20602  | 0.61881  | 0.135571   | 0.262167  |       0.641316  |   0.2543    |       5 |      7.48209 |
| ATP1B3 | activated CD8-positive, alpha-beta T cell, human        |      1.92137 |         1433 |       0.274784    |          0.00175531  |               3919 | 0.801112    | 0.435294 | 0.631395 |  -1.89072  | 0.689416 | 0.169197   | 0.352657  |       0.343931  |   0.256096  |       4 |      7.45419 |
| MAF    | activated CD8-positive, alpha-beta T cell, human        |      1.80023 |          474 |       0.0908917   |          0.000580611 |               3919 | 0.411183    | 0.435294 | 0.631395 |  -1.89072  | 0.689416 | 0.169197   | 0.352657  |       0.343931  |   0.256096  |       4 |      7.45419 |
| PLAU   | activated CD8-positive, alpha-beta T cell, human        |      1.6617  |            2 |       0.000383509 |          2.44983e-06 |               3919 | 0.161733    | 0.435294 | 0.631395 |  -1.89072  | 0.689416 | 0.169197   | 0.352657  |       0.343931  |   0.256096  |       4 |      7.45419 |
| TXNIP  | activated CD8-positive, alpha-beta T cell, human        |      2.15311 |         2010 |       0.385427    |          0.00246208  |               3919 | 1.15155     | 0.435294 | 0.631395 |  -1.89072  | 0.689416 | 0.169197   | 0.352657  |       0.343931  |   0.256096  |       4 |      7.45419 |
| ATP1B3 | naive T cell                                            |      2.04124 |         2656 |       0.259705    |          0.00325338  |               5783 | 0.898748    | 0.338543 | 0.557873 |  -4.05766  | 0.606846 | 0.0984812  | 0.246834  |       0.135836  |   0.198755  |       4 |      7.42239 |
| MAF    | naive T cell                                            |      1.82434 |          506 |       0.0494769   |          0.000619808 |               5783 | 0.367497    | 0.338543 | 0.557873 |  -4.05766  | 0.606846 | 0.0984812  | 0.246834  |       0.135836  |   0.198755  |       4 |      7.42239 |
| PLAU   | naive T cell                                            |      1.69035 |            2 |       0.000344412 |          2.44983e-06 |               5783 | 0.182565    | 0.338543 | 0.557873 |  -4.05766  | 0.606846 | 0.0984812  | 0.246834  |       0.135836  |   0.198755  |       4 |      7.42239 |
| TXNIP  | naive T cell                                            |      1.89086 |         2619 |       0.256087    |          0.00320806  |               5783 | 0.782682    | 0.338543 | 0.557873 |  -4.05766  | 0.606846 | 0.0984812  | 0.246834  |       0.135836  |   0.198755  |       4 |      7.42239 |
| ATP1B3 | activated CD8-positive, alpha-beta T cell               |      1.92898 |         1468 |       0.261303    |          0.00179818  |               4041 | 0.787378    | 0.426328 | 0.624341 |  -1.7715   | 0.682845 | 0.162643   | 0.344236  |       0.35538   |   0.250594  |       4 |      7.38778 |
| MAF    | activated CD8-positive, alpha-beta T cell               |      1.81226 |          486 |       0.0865077   |          0.00059531  |               4041 | 0.413703    | 0.426328 | 0.624341 |  -1.7715   | 0.682845 | 0.162643   | 0.344236  |       0.35538   |   0.250594  |       4 |      7.38778 |
| PLAU   | activated CD8-positive, alpha-beta T cell               |      1.6617  |            2 |       0.000383509 |          2.44983e-06 |               4041 | 0.161733    | 0.426328 | 0.624341 |  -1.7715   | 0.682845 | 0.162643   | 0.344236  |       0.35538   |   0.250594  |       4 |      7.38778 |
| TXNIP  | activated CD8-positive, alpha-beta T cell               |      2.15644 |         2085 |       0.371129    |          0.00255395  |               4041 | 1.13455     | 0.426328 | 0.624341 |  -1.7715   | 0.682845 | 0.162643   | 0.344236  |       0.35538   |   0.250594  |       4 |      7.38778 |
| ATP1B3 | CD4-positive, alpha-beta T cell                         |      1.987   |         3991 |       0.24815     |          0.00488864  |              12233 | 0.877731    | 0.445439 | 0.665728 |   1.41563  | 0.669101 | 0.176612   | 0.32662   |       0.661445  |   0.282873  |       4 |      7.24263 |
| MAF    | CD4-positive, alpha-beta T cell                         |      1.86248 |         2945 |       0.182987    |          0.00360738  |              12233 | 0.661095    | 0.445439 | 0.665728 |   1.41563  | 0.669101 | 0.176612   | 0.32662   |       0.661445  |   0.282873  |       4 |      7.24263 |
| PLAU   | CD4-positive, alpha-beta T cell                         |      1.50567 |            2 |       0.000313381 |          2.44983e-06 |              12233 | 0.0478545   | 0.445439 | 0.665728 |   1.41563  | 0.669101 | 0.176612   | 0.32662   |       0.661445  |   0.282873  |       4 |      7.24263 |
| TXNIP  | CD4-positive, alpha-beta T cell                         |      2.04486 |         5295 |       0.329005    |          0.00648593  |              12233 | 1.07623     | 0.445439 | 0.665728 |   1.41563  | 0.669101 | 0.176612   | 0.32662   |       0.661445  |   0.282873  |       4 |      7.24263 |
| ATP1B3 | T follicular helper cell                                |      2.11603 |         1446 |       0.471163    |          0.00177123  |               3029 | 1.23796     | 0.509294 | 0.681895 |   0.871406 | 0.746879 | 0.223283   | 0.426304  |       0.609183  |   0.295482  |       4 |      7.23156 |
| MAF    | T follicular helper cell                                |      1.89576 |          991 |       0.322906    |          0.00121389  |               3029 | 0.842747    | 0.509294 | 0.681895 |   0.871406 | 0.746879 | 0.223283   | 0.426304  |       0.609183  |   0.295482  |       4 |      7.23156 |
| PLAU   | T follicular helper cell                                |      1.46531 |            1 |       0.000325839 |          1.22492e-06 |               3029 | 0.0184218   | 0.509294 | 0.681895 |   0.871406 | 0.746879 | 0.223283   | 0.426304  |       0.609183  |   0.295482  |       4 |      7.23156 |
| TXNIP  | T follicular helper cell                                |      1.88472 |          591 |       0.192571    |          0.000723926 |               3029 | 0.628448    | 0.509294 | 0.681895 |   0.871406 | 0.746879 | 0.223283   | 0.426304  |       0.609183  |   0.295482  |       4 |      7.23156 |
| ATP1B3 | CL:0000920                                              |      2.17565 |         4191 |       0.500537    |          0.00513363  |               7632 | 1.39924     | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| MAF    | CL:0000920                                              |      1.95094 |         1503 |       0.179506    |          0.00184105  |               7632 | 0.681639    | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| PLAU   | CL:0000920                                              |      1.45264 |            1 |       0.000119432 |          1.22492e-06 |               7632 | 0.00887397  | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| TXNIP  | CL:0000920                                              |      1.90187 |         1937 |       0.231339    |          0.00237266  |               7632 | 0.735261    | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| ATP1B3 | CD8-positive, alpha-beta regulatory T cell              |      2.17565 |         4191 |       0.500537    |          0.00513363  |               7632 | 1.39924     | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| MAF    | CD8-positive, alpha-beta regulatory T cell              |      1.95094 |         1503 |       0.179506    |          0.00184105  |               7632 | 0.681639    | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| PLAU   | CD8-positive, alpha-beta regulatory T cell              |      1.45264 |            1 |       0.000119432 |          1.22492e-06 |               7632 | 0.00887397  | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| TXNIP  | CD8-positive, alpha-beta regulatory T cell              |      1.90187 |         1937 |       0.231339    |          0.00237266  |               7632 | 0.735261    | 0.568043 | 0.706254 |   1.45581  | 0.804304 | 0.266223   | 0.499901  |       0.665304  |   0.31448   |       4 |      7.15625 |
| ATP1B3 | CD4-positive helper T cell                              |      2.06818 |         2053 |       0.426908    |          0.00251475  |               4492 | 1.153       | 0.473528 | 0.654313 |   1.46573  | 0.723702 | 0.197142   | 0.3966    |       0.666256  |   0.27397   |       4 |      6.94339 |
| MAF    | CD4-positive helper T cell                              |      1.87842 |         1403 |       0.291745    |          0.00171856  |               4492 | 0.794427    | 0.473528 | 0.654313 |   1.46573  | 0.723702 | 0.197142   | 0.3966    |       0.666256  |   0.27397   |       4 |      6.94339 |
| PLAU   | CD4-positive helper T cell                              |      1.46531 |            1 |       0.000325839 |          1.22492e-06 |               4492 | 0.0184218   | 0.473528 | 0.654313 |   1.46573  | 0.723702 | 0.197142   | 0.3966    |       0.666256  |   0.27397   |       4 |      6.94339 |
| TXNIP  | CD4-positive helper T cell                              |      1.85325 |         1035 |       0.215221    |          0.00126779  |               4492 | 0.651404    | 0.473528 | 0.654313 |   1.46573  | 0.723702 | 0.197142   | 0.3966    |       0.666256  |   0.27397   |       4 |      6.94339 |
| ATP1B3 | Langerhans cell                                         |      1.94776 |         4770 |       0.382487    |          0.00584285  |               5450 | 1.07155     | 0.280398 | 0.570717 |   4.94107  | 0.491308 | 0.0559828  | 0.0987566 |       1         |   0.208772  |       5 |      6.61333 |
| MAF    | Langerhans cell                                         |      2.0582  |           10 |       0.00080186  |          1.22492e-05 |               5450 | 0.451678    | 0.280398 | 0.570717 |   4.94107  | 0.491308 | 0.0559828  | 0.0987566 |       1         |   0.208772  |       5 |      6.61333 |
| MMP13  | Langerhans cell                                         |      2.07237 |            5 |       0.000402156 |          6.12458e-06 |               5450 | 0.461277    | 0.280398 | 0.570717 |   4.94107  | 0.491308 | 0.0559828  | 0.0987566 |       1         |   0.208772  |       5 |      6.61333 |
| PLAU   | Langerhans cell                                         |      1.99192 |          123 |       0.00989303  |          0.000150665 |               5450 | 0.420023    | 0.280398 | 0.570717 |   4.94107  | 0.491308 | 0.0559828  | 0.0987566 |       1         |   0.208772  |       5 |      6.61333 |
| TXNIP  | Langerhans cell                                         |      1.94724 |          542 |       0.0434608   |          0.000663905 |               5450 | 0.449058    | 0.280398 | 0.570717 |   4.94107  | 0.491308 | 0.0559828  | 0.0987566 |       1         |   0.208772  |       5 |      6.61333 |
| ATP1B3 | smooth muscle cell                                      |      1.89532 |         1478 |       0.146598    |          0.00181043  |               3365 | 0.591045    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| MAF    | smooth muscle cell                                      |      1.60786 |          142 |       0.0140845   |          0.000173938 |               3365 | 0.146785    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| MMP13  | smooth muscle cell                                      |      1.6824  |           18 |       0.0269461   |          2.20485e-05 |               3365 | 0.217101    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| PLAU   | smooth muscle cell                                      |      2.2642  |          123 |       0.0122      |          0.000150665 |               3365 | 0.622015    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| TXNIP  | smooth muscle cell                                      |      1.81684 |         1604 |       0.159095    |          0.00196477  |               3365 | 0.555953    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| ATP1B3 | muscle cell                                             |      1.89532 |         1478 |       0.146598    |          0.00181043  |               3365 | 0.591045    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| MAF    | muscle cell                                             |      1.60786 |          142 |       0.0140845   |          0.000173938 |               3365 | 0.146785    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| MMP13  | muscle cell                                             |      1.6824  |           18 |       0.0269461   |          2.20485e-05 |               3365 | 0.217101    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| PLAU   | muscle cell                                             |      2.2642  |          123 |       0.0122      |          0.000150665 |               3365 | 0.622015    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| TXNIP  | muscle cell                                             |      1.81684 |         1604 |       0.159095    |          0.00196477  |               3365 | 0.555953    | 0.225913 | 0.42658  |  -2.94928  | 0.529592 | 0.0161595  | 0.147822  |       0.242276  |   0.0963565 |       5 |      6.53648 |
| ATP1B3 | contractile cell                                        |      1.91592 |         4068 |       0.120945    |          0.00498296  |              11338 | 0.637167    | 0.269866 | 0.453807 |  -1.99726  | 0.594671 | 0.0482849  | 0.23123   |       0.3337    |   0.117592  |       5 |      6.41986 |
| MAF    | contractile cell                                        |      1.6544  |          358 |       0.0106437   |          0.00043852  |              11338 | 0.181365    | 0.269866 | 0.453807 |  -1.99726  | 0.594671 | 0.0482849  | 0.23123   |       0.3337    |   0.117592  |       5 |      6.41986 |
| MMP13  | contractile cell                                        |      1.66772 |          414 |       0.0222964   |          0.000507116 |              11338 | 0.210055    | 0.269866 | 0.453807 |  -1.99726  | 0.594671 | 0.0482849  | 0.23123   |       0.3337    |   0.117592  |       5 |      6.41986 |
| PLAU   | contractile cell                                        |      1.9961  |          420 |       0.012487    |          0.000514465 |              11338 | 0.43494     | 0.269866 | 0.453807 |  -1.99726  | 0.594671 | 0.0482849  | 0.23123   |       0.3337    |   0.117592  |       5 |      6.41986 |
| TXNIP  | contractile cell                                        |      1.94981 |         6078 |       0.180705    |          0.00744504  |              11338 | 0.80551     | 0.269866 | 0.453807 |  -1.99726  | 0.594671 | 0.0482849  | 0.23123   |       0.3337    |   0.117592  |       5 |      6.41986 |
| ATP1B3 | melanocyte                                              |      1.87432 |         4143 |       0.342737    |          0.00507483  |               7124 | 0.941537    | 0.318934 | 0.480748 |  -0.939329 | 0.663411 | 0.0841484  | 0.319329  |       0.435295  |   0.138603  |       5 |      6.2629  |
| MAF    | melanocyte                                              |      1.79706 |          456 |       0.0377234   |          0.000558562 |               7124 | 0.328638    | 0.318934 | 0.480748 |  -0.939329 | 0.663411 | 0.0841484  | 0.319329  |       0.435295  |   0.138603  |       5 |      6.2629  |
| MMP13  | melanocyte                                              |      1.72212 |          154 |       0.0604396   |          0.000188637 |               7124 | 0.299954    | 0.318934 | 0.480748 |  -0.939329 | 0.663411 | 0.0841484  | 0.319329  |       0.435295  |   0.138603  |       5 |      6.2629  |
| PLAU   | melanocyte                                              |      1.65413 |           38 |       0.00314361  |          4.65468e-05 |               7124 | 0.16132     | 0.318934 | 0.480748 |  -0.939329 | 0.663411 | 0.0841484  | 0.319329  |       0.435295  |   0.138603  |       5 |      6.2629  |
| TXNIP  | melanocyte                                              |      1.87978 |         2333 |       0.193001    |          0.00285773  |               7124 | 0.672292    | 0.318934 | 0.480748 |  -0.939329 | 0.663411 | 0.0841484  | 0.319329  |       0.435295  |   0.138603  |       5 |      6.2629  |
| ATP1B3 | supporting cell                                         |      1.93929 |         2362 |       0.131955    |          0.00289325  |               4997 | 0.624893    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MAF    | supporting cell                                         |      1.68149 |          153 |       0.00854749  |          0.000187412 |               4997 | 0.192463    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MMP13  | supporting cell                                         |      1.66705 |          396 |       0.0221229   |          0.000485067 |               4997 | 0.208825    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| PLAU   | supporting cell                                         |      1.88096 |          264 |       0.0147486   |          0.000323378 |               4997 | 0.350194    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| TXNIP  | supporting cell                                         |      1.9649  |         1822 |       0.101788    |          0.0022318   |               4997 | 0.583806    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| ATP1B3 | pericyte                                                |      1.93929 |         2362 |       0.131955    |          0.00289325  |               4997 | 0.624893    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MAF    | pericyte                                                |      1.68149 |          153 |       0.00854749  |          0.000187412 |               4997 | 0.192463    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MMP13  | pericyte                                                |      1.66705 |          396 |       0.0221229   |          0.000485067 |               4997 | 0.208825    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| PLAU   | pericyte                                                |      1.88096 |          264 |       0.0147486   |          0.000323378 |               4997 | 0.350194    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| TXNIP  | pericyte                                                |      1.9649  |         1822 |       0.101788    |          0.0022318   |               4997 | 0.583806    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| ATP1B3 | mural cell                                              |      1.93929 |         2362 |       0.131955    |          0.00289325  |               4997 | 0.624893    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MAF    | mural cell                                              |      1.68149 |          153 |       0.00854749  |          0.000187412 |               4997 | 0.192463    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| MMP13  | mural cell                                              |      1.66705 |          396 |       0.0221229   |          0.000485067 |               4997 | 0.208825    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| PLAU   | mural cell                                              |      1.88096 |          264 |       0.0147486   |          0.000323378 |               4997 | 0.350194    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| TXNIP  | mural cell                                              |      1.9649  |         1822 |       0.101788    |          0.0022318   |               4997 | 0.583806    | 0.203804 | 0.392036 |  -2.922    | 0.519861 | 0          | 0.135351  |       0.244895  |   0.069415  |       5 |      6.21132 |
| ATP1B3 | neuron associated cell                                  |      2.81218 |         1473 |       0.234891    |          0.0018043   |               3269 | 1.39188     | 0.52548  | 0.57297  |   0.215921 | 0.917117 | 0.235114   | 0.644486  |       0.546235  |   0.210529  |       5 |      6.09574 |
| MAF    | neuron associated cell                                  |      1.67543 |           54 |       0.0105243   |          6.61455e-05 |               3269 | 0.188353    | 0.52548  | 0.57297  |   0.215921 | 0.917117 | 0.235114   | 0.644486  |       0.546235  |   0.210529  |       5 |      6.09574 |
| MMP13  | neuron associated cell                                  |      1.7049  |           35 |       0.0185087   |          4.28721e-05 |               3269 | 0.221304    | 0.52548  | 0.57297  |   0.215921 | 0.917117 | 0.235114   | 0.644486  |       0.546235  |   0.210529  |       5 |      6.09574 |
| PLAU   | neuron associated cell                                  |      1.75889 |           67 |       0.0106841   |          8.20694e-05 |               3269 | 0.249793    | 0.52548  | 0.57297  |   0.215921 | 0.917117 | 0.235114   | 0.644486  |       0.546235  |   0.210529  |       5 |      6.09574 |
| TXNIP  | neuron associated cell                                  |      1.95804 |         1640 |       0.261521    |          0.00200886  |               3269 | 0.813519    | 0.52548  | 0.57297  |   0.215921 | 0.917117 | 0.235114   | 0.644486  |       0.546235  |   0.210529  |       5 |      6.09574 |
| ATP1B3 | cytotoxic T cell                                        |      1.78991 |         2151 |       0.192157    |          0.0026348   |               7108 | 0.600602    | 0.45946  | 0.582307 |   3.17297  | 0.789034 | 0.18686    | 0.480331  |       0.830206  |   0.217811  |       5 |      6.05457 |
| MAF    | cytotoxic T cell                                        |      1.84153 |          260 |       0.0232267   |          0.000318478 |               7108 | 0.33405     | 0.45946  | 0.582307 |   3.17297  | 0.789034 | 0.18686    | 0.480331  |       0.830206  |   0.217811  |       5 |      6.05457 |
| MMP13  | cytotoxic T cell                                        |      1.97821 |           55 |       0.00491335  |          6.73704e-05 |               7108 | 0.400734    | 0.45946  | 0.582307 |   3.17297  | 0.789034 | 0.18686    | 0.480331  |       0.830206  |   0.217811  |       5 |      6.05457 |
| PLAU   | cytotoxic T cell                                        |      1.72557 |           24 |       0.00214401  |          2.9398e-05  |               7108 | 0.211534    | 0.45946  | 0.582307 |   3.17297  | 0.789034 | 0.18686    | 0.480331  |       0.830206  |   0.217811  |       5 |      6.05457 |
| TXNIP  | cytotoxic T cell                                        |      2.29346 |         4618 |       0.412542    |          0.00565667  |               7108 | 1.36461     | 0.45946  | 0.582307 |   3.17297  | 0.789034 | 0.18686    | 0.480331  |       0.830206  |   0.217811  |       5 |      6.05457 |
| ATP1B3 | central memory CD8-positive, alpha-beta T cell          |      2.07155 |         8361 |       0.484415    |          0.0102415   |              13275 | 1.41117     | 0.531176 | 0.579303 |   0.942431 | 0.916923 | 0.239277   | 0.644237  |       0.616003  |   0.215468  |       5 |      6.01752 |
| MAF    | central memory CD8-positive, alpha-beta T cell          |      1.8212  |         1359 |       0.078737    |          0.00166466  |              13275 | 0.432014    | 0.531176 | 0.579303 |   0.942431 | 0.916923 | 0.239277   | 0.644237  |       0.616003  |   0.215468  |       5 |      6.01752 |
| MMP13  | central memory CD8-positive, alpha-beta T cell          |      1.78719 |            1 |       5.79374e-05 |          1.22492e-06 |              13275 | 0.252717    | 0.531176 | 0.579303 |   0.942431 | 0.916923 | 0.239277   | 0.644237  |       0.616003  |   0.215468  |       5 |      6.01752 |
| PLAU   | central memory CD8-positive, alpha-beta T cell          |      1.50932 |            8 |       0.000463499 |          9.79933e-06 |              13275 | 0.050907    | 0.531176 | 0.579303 |   0.942431 | 0.916923 | 0.239277   | 0.644237  |       0.616003  |   0.215468  |       5 |      6.01752 |
| TXNIP  | central memory CD8-positive, alpha-beta T cell          |      1.91567 |         3546 |       0.205446    |          0.00434355  |              13275 | 0.749708    | 0.531176 | 0.579303 |   0.942431 | 0.916923 | 0.239277   | 0.644237  |       0.616003  |   0.215468  |       5 |      6.01752 |
| ATP1B3 | perivascular cell                                       |      1.86761 |         3491 |       0.12781     |          0.00427618  |               7845 | 0.596734    | 0.208547 | 0.377286 |  -3.06226  | 0.552757 | 0.0034665  | 0.177511  |       0.231426  |   0.0579108 |       5 |      5.98594 |
| MAF    | perivascular cell                                       |      1.6434  |          286 |       0.0104708   |          0.000350326 |               7845 | 0.171149    | 0.208547 | 0.377286 |  -3.06226  | 0.552757 | 0.0034665  | 0.177511  |       0.231426  |   0.0579108 |       5 |      5.98594 |
| MMP13  | perivascular cell                                       |      1.66705 |          396 |       0.0221229   |          0.000485067 |               7845 | 0.208825    | 0.208547 | 0.377286 |  -3.06226  | 0.552757 | 0.0034665  | 0.177511  |       0.231426  |   0.0579108 |       5 |      5.98594 |
| PLAU   | perivascular cell                                       |      1.82771 |          321 |       0.0117522   |          0.000393198 |               7845 | 0.308396    | 0.208547 | 0.377286 |  -3.06226  | 0.552757 | 0.0034665  | 0.177511  |       0.231426  |   0.0579108 |       5 |      5.98594 |
| TXNIP  | perivascular cell                                       |      1.8896  |         3351 |       0.122684    |          0.0041047   |               7845 | 0.601323    | 0.208547 | 0.377286 |  -3.06226  | 0.552757 | 0.0034665  | 0.177511  |       0.231426  |   0.0579108 |       5 |      5.98594 |
| ATP1B3 | memory T cell                                           |      2.06932 |         8423 |       0.305026    |          0.0103175   |              14405 | 1.14213     | 0.406097 | 0.51241  |   0.42835  | 0.792523 | 0.147857   | 0.484803  |       0.566635  |   0.163298  |       5 |      5.88758 |
| MAF    | memory T cell                                           |      1.81529 |         1497 |       0.05419     |          0.0018337   |              14405 | 0.394591    | 0.406097 | 0.51241  |   0.42835  | 0.792523 | 0.147857   | 0.484803  |       0.566635  |   0.163298  |       5 |      5.88758 |
| MMP13  | memory T cell                                           |      1.67738 |          218 |       0.00789455  |          0.000267032 |              14405 | 0.190236    | 0.406097 | 0.51241  |   0.42835  | 0.792523 | 0.147857   | 0.484803  |       0.566635  |   0.163298  |       5 |      5.88758 |
| PLAU   | memory T cell                                           |      1.66187 |           48 |       0.00173825  |          5.8796e-05  |              14405 | 0.165124    | 0.406097 | 0.51241  |   0.42835  | 0.792523 | 0.147857   | 0.484803  |       0.566635  |   0.163298  |       5 |      5.88758 |
| TXNIP  | memory T cell                                           |      1.88997 |         4219 |       0.152724    |          0.00516792  |              14405 | 0.669969    | 0.406097 | 0.51241  |   0.42835  | 0.792523 | 0.147857   | 0.484803  |       0.566635  |   0.163298  |       5 |      5.88758 |
| ATP1B3 | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      2.06932 |         8423 |       0.305026    |          0.0103175   |              14397 | 1.14213     | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| MAF    | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.8146  |         1496 |       0.0541754   |          0.00183248  |              14397 | 0.394043    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| MMP13  | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.67738 |          218 |       0.00789455  |          0.000267032 |              14397 | 0.190236    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| PLAU   | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.66187 |           48 |       0.00173825  |          5.8796e-05  |              14397 | 0.165124    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| TXNIP  | CD8-positive, alpha-beta memory T cell, CD45RO-positive |      1.88834 |         4212 |       0.152531    |          0.00515935  |              14397 | 0.668302    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| ATP1B3 | CD8-positive, alpha-beta memory T cell                  |      2.06932 |         8423 |       0.305026    |          0.0103175   |              14397 | 1.14213     | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| MAF    | CD8-positive, alpha-beta memory T cell                  |      1.8146  |         1496 |       0.0541754   |          0.00183248  |              14397 | 0.394043    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| MMP13  | CD8-positive, alpha-beta memory T cell                  |      1.67738 |          218 |       0.00789455  |          0.000267032 |              14397 | 0.190236    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| PLAU   | CD8-positive, alpha-beta memory T cell                  |      1.66187 |           48 |       0.00173825  |          5.8796e-05  |              14397 | 0.165124    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| TXNIP  | CD8-positive, alpha-beta memory T cell                  |      1.88834 |         4212 |       0.152531    |          0.00515935  |              14397 | 0.668302    | 0.405976 | 0.511967 |   0.45185  | 0.792972 | 0.147768   | 0.485378  |       0.568892  |   0.162952  |       5 |      5.87725 |
| ATP1B3 | monocyte                                                |      1.77597 |         1361 |       0.153369    |          0.00166711  |               3916 | 0.511041    | 0.239436 | 0.435089 |   1.02949  | 0.550314 | 0.026043   | 0.17438   |       0.624364  |   0.102993  |       5 |      5.79232 |
| MAF    | monocyte                                                |      1.79885 |          551 |       0.0620915   |          0.000674929 |               3916 | 0.369046    | 0.239436 | 0.435089 |   1.02949  | 0.550314 | 0.026043   | 0.17438   |       0.624364  |   0.102993  |       5 |      5.79232 |
| MMP13  | monocyte                                                |      1.64097 |           23 |       0.00259184  |          2.81731e-05 |               3916 | 0.150491    | 0.239436 | 0.435089 |   1.02949  | 0.550314 | 0.026043   | 0.17438   |       0.624364  |   0.102993  |       5 |      5.79232 |
| PLAU   | monocyte                                                |      1.86552 |          197 |       0.0221997   |          0.000241309 |               3916 | 0.348308    | 0.239436 | 0.435089 |   1.02949  | 0.550314 | 0.026043   | 0.17438   |       0.624364  |   0.102993  |       5 |      5.79232 |
| TXNIP  | monocyte                                                |      2.05391 |         1784 |       0.201037    |          0.00218525  |               3916 | 0.796559    | 0.239436 | 0.435089 |   1.02949  | 0.550314 | 0.026043   | 0.17438   |       0.624364  |   0.102993  |       5 |      5.79232 |
| ATP1B3 | helper T cell                                           |      1.78925 |         3660 |       0.0858107   |          0.0044832   |              24501 | 0.481139    | 0.797813 | 0.671921 |   4.11822  | 1.18736  | 0.434163   | 0.990838  |       0.92098   |   0.287703  |       5 |      5.33589 |
| MAF    | helper T cell                                           |      1.88059 |         2960 |       0.0693989   |          0.00362575  |              24501 | 0.504322    | 0.797813 | 0.671921 |   4.11822  | 1.18736  | 0.434163   | 0.990838  |       0.92098   |   0.287703  |       5 |      5.33589 |
| MMP13  | helper T cell                                           |      1.63432 |          425 |       0.00996436  |          0.00052059  |              24501 | 0.1675      | 0.797813 | 0.671921 |   4.11822  | 1.18736  | 0.434163   | 0.990838  |       0.92098   |   0.287703  |       5 |      5.33589 |
| PLAU   | helper T cell                                           |      1.6272  |           70 |       0.00164119  |          8.57442e-05 |              24501 | 0.140287    | 0.797813 | 0.671921 |   4.11822  | 1.18736  | 0.434163   | 0.990838  |       0.92098   |   0.287703  |       5 |      5.33589 |
| TXNIP  | helper T cell                                           |      2.79559 |        17386 |       0.407624    |          0.0212964   |              24501 | 2.06636     | 0.797813 | 0.671921 |   4.11822  | 1.18736  | 0.434163   | 0.990838  |       0.92098   |   0.287703  |       5 |      5.33589 |
| ATP1B3 | dendritic cell                                          |      1.93541 |        14140 |       0.329527    |          0.0173203   |              18367 | 1.23481     | 0.420232 | 0.510313 |   3.55749  | 0.823479 | 0.158188   | 0.524477  |       0.867132  |   0.161662  |       5 |      5.16367 |
| MAF    | dendritic cell                                          |      1.75742 |          405 |       0.00968205  |          0.000496091 |              18367 | 0.256297    | 0.420232 | 0.510313 |   3.55749  | 0.823479 | 0.158188   | 0.524477  |       0.867132  |   0.161662  |       5 |      5.16367 |
| MMP13  | dendritic cell                                          |      1.72412 |           95 |       0.00227316  |          0.000116367 |              18367 | 0.212575    | 0.420232 | 0.510313 |   3.55749  | 0.823479 | 0.158188   | 0.524477  |       0.867132  |   0.161662  |       5 |      5.16367 |
| PLAU   | dendritic cell                                          |      1.86951 |          477 |       0.0111261   |          0.000584285 |              18367 | 0.342131    | 0.420232 | 0.510313 |   3.55749  | 0.823479 | 0.158188   | 0.524477  |       0.867132  |   0.161662  |       5 |      5.16367 |
| TXNIP  | dendritic cell                                          |      1.85882 |         3250 |       0.0757399   |          0.00398098  |              18367 | 0.505749    | 0.420232 | 0.510313 |   3.55749  | 0.823479 | 0.158188   | 0.524477  |       0.867132  |   0.161662  |       5 |      5.16367 |
| ATP1B3 | B cell                                                  |      1.79863 |         1215 |       0.13749     |          0.00148827  |               4946 | 0.49982     | 0.470932 | 0.499573 |   1.49511  | 0.942669 | 0.195245   | 0.677234  |       0.669078  |   0.153286  |       4 |      4.80529 |
| MAF    | B cell                                                  |      1.62808 |            5 |       0.00295334  |          6.12458e-06 |               4946 | 0.141153    | 0.470932 | 0.499573 |   1.49511  | 0.942669 | 0.195245   | 0.677234  |       0.669078  |   0.153286  |       4 |      4.80529 |
| PLAU   | B cell                                                  |      1.70223 |            9 |       0.00106045  |          1.10243e-05 |               4946 | 0.19249     | 0.470932 | 0.499573 |   1.49511  | 0.942669 | 0.195245   | 0.677234  |       0.669078  |   0.153286  |       4 |      4.80529 |
| TXNIP  | B cell                                                  |      2.03604 |         3717 |       0.420618    |          0.00455302  |               4946 | 1.16483     | 0.470932 | 0.499573 |   1.49511  | 0.942669 | 0.195245   | 0.677234  |       0.669078  |   0.153286  |       4 |      4.80529 |
| ATP1B3 | conventional dendritic cell                             |      1.94525 |        11522 |       0.35806     |          0.0141135   |              14328 | 1.21446     | 0.420846 | 0.484911 |   3.74973  | 0.867882 | 0.158636   | 0.581384  |       0.885593  |   0.14185   |       5 |      4.73468 |
| MAF    | conventional dendritic cell                             |      1.7256  |           67 |       0.00208911  |          8.20694e-05 |              14328 | 0.212628    | 0.420846 | 0.484911 |   3.74973  | 0.867882 | 0.158636   | 0.581384  |       0.885593  |   0.14185   |       5 |      4.73468 |
| MMP13  | conventional dendritic cell                             |      1.73032 |           73 |       0.0022789   |          8.94189e-05 |              14328 | 0.216513    | 0.420846 | 0.484911 |   3.74973  | 0.867882 | 0.158636   | 0.581384  |       0.885593  |   0.14185   |       5 |      4.73468 |
| PLAU   | conventional dendritic cell                             |      1.84683 |          222 |       0.00690707  |          0.000271932 |              14328 | 0.31241     | 0.420846 | 0.484911 |   3.74973  | 0.867882 | 0.158636   | 0.581384  |       0.885593  |   0.14185   |       5 |      4.73468 |
| TXNIP  | conventional dendritic cell                             |      1.83706 |         2444 |       0.0759502   |          0.0029937   |              14328 | 0.468542    | 0.420846 | 0.484911 |   3.74973  | 0.867882 | 0.158636   | 0.581384  |       0.885593  |   0.14185   |       5 |      4.73468 |
| ATP1B3 | vascular associated smooth muscle cell                  |      1.71763 |         1129 |       0.119928    |          0.00138293  |               2848 | 0.412104    | 0.211142 | 0.303033 |  -4.0041   | 0.696763 | 0.00536349 | 0.362073  |       0.14098   |   0         |       4 |      4.72963 |
| MAF    | vascular associated smooth muscle cell                  |      1.59959 |          133 |       0.0141279   |          0.000162914 |               2848 | 0.140577    | 0.211142 | 0.303033 |  -4.0041   | 0.696763 | 0.00536349 | 0.362073  |       0.14098   |   0         |       4 |      4.72963 |
| PLAU   | vascular associated smooth muscle cell                  |      1.58103 |           57 |       0.00605481  |          6.98203e-05 |               2848 | 0.112897    | 0.211142 | 0.303033 |  -4.0041   | 0.696763 | 0.00536349 | 0.362073  |       0.14098   |   0         |       4 |      4.72963 |
| TXNIP  | vascular associated smooth muscle cell                  |      1.79988 |         1529 |       0.162418    |          0.0018729   |               2848 | 0.546555    | 0.211142 | 0.303033 |  -4.0041   | 0.696763 | 0.00536349 | 0.362073  |       0.14098   |   0         |       4 |      4.72963 |
| ATP1B3 | innate lymphoid cell                                    |      1.80577 |         2815 |       0.0917655   |          0.00344814  |              12556 | 0.479418    | 0.415358 | 0.469864 |   3.39929  | 0.883997 | 0.154625   | 0.602039  |       0.85194   |   0.130114  |       5 |      4.60439 |
| MAF    | innate lymphoid cell                                    |      1.7271  |         1117 |       0.0364128   |          0.00136823  |              12556 | 0.293411    | 0.415358 | 0.469864 |   3.39929  | 0.883997 | 0.154625   | 0.602039  |       0.85194   |   0.130114  |       5 |      4.60439 |
| MMP13  | innate lymphoid cell                                    |      1.67761 |          503 |       0.0175598   |          0.000616133 |              12556 | 0.212551    | 0.415358 | 0.469864 |   3.39929  | 0.883997 | 0.154625   | 0.602039  |       0.85194   |   0.130114  |       5 |      4.60439 |
| PLAU   | innate lymphoid cell                                    |      1.67664 |          114 |       0.00371626  |          0.000139641 |              12556 | 0.180632    | 0.415358 | 0.469864 |   3.39929  | 0.883997 | 0.154625   | 0.602039  |       0.85194   |   0.130114  |       5 |      4.60439 |
| TXNIP  | innate lymphoid cell                                    |      2.23165 |         8007 |       0.261018    |          0.00980791  |              12556 | 1.1833      | 0.415358 | 0.469864 |   3.39929  | 0.883997 | 0.154625   | 0.602039  |       0.85194   |   0.130114  |       5 |      4.60439 |
| ATP1B3 | myofibroblast cell                                      |      1.73399 |          203 |       0.0366691   |          0.000248658 |               2931 | 0.274266    | 0.47762  | 0.498901 |   3.52146  | 0.957343 | 0.200133   | 0.696041  |       0.863672  |   0.152761  |       4 |      4.38396 |
| MAF    | myofibroblast cell                                      |      1.64368 |           57 |       0.0102962   |          6.98203e-05 |               2931 | 0.164935    | 0.47762  | 0.498901 |   3.52146  | 0.957343 | 0.200133   | 0.696041  |       0.863672  |   0.152761  |       4 |      4.38396 |
| PLAU   | myofibroblast cell                                      |      1.90799 |           32 |       0.00578035  |          3.91973e-05 |               2931 | 0.350213    | 0.47762  | 0.498901 |   3.52146  | 0.957343 | 0.200133   | 0.696041  |       0.863672  |   0.152761  |       4 |      4.38396 |
| TXNIP  | myofibroblast cell                                      |      2.01711 |         2639 |       0.476698    |          0.00323256  |               2931 | 1.20619     | 0.47762  | 0.498901 |   3.52146  | 0.957343 | 0.200133   | 0.696041  |       0.863672  |   0.152761  |       4 |      4.38396 |
| ATP1B3 | lymphocyte of B lineage                                 |      1.79415 |         1286 |       0.10714     |          0.00157524  |               5391 | 0.452933    | 0.370148 | 0.42898  |   2.9191   | 0.862857 | 0.121581   | 0.574945  |       0.805827  |   0.0982283 |       5 |      4.37049 |
| MAF    | lymphocyte of B lineage                                 |      1.62569 |            7 |       0.00144063  |          8.57442e-06 |               5391 | 0.137192    | 0.370148 | 0.42898  |   2.9191   | 0.862857 | 0.121581   | 0.574945  |       0.805827  |   0.0982283 |       5 |      4.37049 |
| MMP13  | lymphocyte of B lineage                                 |      1.85698 |            8 |       0.00252685  |          9.79933e-06 |               5391 | 0.307494    | 0.370148 | 0.42898  |   2.9191   | 0.862857 | 0.121581   | 0.574945  |       0.805827  |   0.0982283 |       5 |      4.37049 |
| PLAU   | lymphocyte of B lineage                                 |      1.70223 |            9 |       0.00106045  |          1.10243e-05 |               5391 | 0.19249     | 0.370148 | 0.42898  |   2.9191   | 0.862857 | 0.121581   | 0.574945  |       0.805827  |   0.0982283 |       5 |      4.37049 |
| TXNIP  | lymphocyte of B lineage                                 |      2.03756 |         4081 |       0.339998    |          0.00499889  |               5391 | 1.05479     | 0.370148 | 0.42898  |   2.9191   | 0.862857 | 0.121581   | 0.574945  |       0.805827  |   0.0982283 |       5 |      4.37049 |
| ATP1B3 | natural killer cell                                     |      1.7757  |         1008 |       0.0543631   |          0.00123472  |               5508 | 0.352849    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| MAF    | natural killer cell                                     |      1.71231 |          653 |       0.0352173   |          0.000799871 |               5508 | 0.268369    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| MMP13  | natural killer cell                                     |      1.67841 |          396 |       0.0213569   |          0.000485067 |               5508 | 0.215958    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| PLAU   | natural killer cell                                     |      1.69045 |           94 |       0.00506957  |          0.000115142 |               5508 | 0.192194    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| TXNIP  | natural killer cell                                     |      2.07958 |         3357 |       0.181048    |          0.00411205  |               5508 | 0.827552    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| ATP1B3 | group 1 innate lymphoid cell                            |      1.7757  |         1008 |       0.0543631   |          0.00123472  |               5508 | 0.352849    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| MAF    | group 1 innate lymphoid cell                            |      1.71231 |          653 |       0.0352173   |          0.000799871 |               5508 | 0.268369    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| MMP13  | group 1 innate lymphoid cell                            |      1.67841 |          396 |       0.0213569   |          0.000485067 |               5508 | 0.215958    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| PLAU   | group 1 innate lymphoid cell                            |      1.69045 |           94 |       0.00506957  |          0.000115142 |               5508 | 0.192194    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| TXNIP  | group 1 innate lymphoid cell                            |      2.07958 |         3357 |       0.181048    |          0.00411205  |               5508 | 0.827552    | 0.26235  | 0.371384 |   3.8814   | 0.706412 | 0.0427915  | 0.37444   |       0.898238  |   0.0533084 |       5 |      4.11538 |
| ATP1B3 | myeloid dendritic cell                                  |      1.94602 |         6592 |       0.531356    |          0.00807465  |               8508 | 1.34253     | 0.5357   | 0.448469 |   2.46243  | 1.19451  | 0.242584   | 1         |       0.761972  |   0.113428  |       5 |      3.57888 |
| MAF    | myeloid dendritic cell                                  |      1.70022 |           11 |       0.000894454 |          1.34741e-05 |               8508 | 0.19083     | 0.5357   | 0.448469 |   2.46243  | 1.19451  | 0.242584   | 1         |       0.761972  |   0.113428  |       5 |      3.57888 |
| MMP13  | myeloid dendritic cell                                  |      1.44072 |            1 |       8.1314e-05  |          1.22492e-06 |               8508 | 0.000121971 | 0.5357   | 0.448469 |   2.46243  | 1.19451  | 0.242584   | 1         |       0.761972  |   0.113428  |       5 |      3.57888 |
| PLAU   | myeloid dendritic cell                                  |      1.66306 |           96 |       0.00773819  |          0.000117592 |               8508 | 0.17628     | 0.5357   | 0.448469 |   2.46243  | 1.19451  | 0.242584   | 1         |       0.761972  |   0.113428  |       5 |      3.57888 |
| TXNIP  | myeloid dendritic cell                                  |      1.80476 |         1808 |       0.145736    |          0.00221465  |               8508 | 0.532584    | 0.5357   | 0.448469 |   2.46243  | 1.19451  | 0.242584   | 1         |       0.761972  |   0.113428  |       5 |      3.57888 |
| ATP1B3 | monocyte-derived dendritic cell                         |      1.93235 |         6506 |       0.529029    |          0.00796931  |               8398 | 1.32676     | 0.529119 | 0.444091 |   2.45008  | 1.19146  | 0.237774   | 0.996098  |       0.760786  |   0.110014  |       5 |      3.55088 |
| MAF    | monocyte-derived dendritic cell                         |      1.70022 |           11 |       0.000894454 |          1.34741e-05 |               8398 | 0.19083     | 0.529119 | 0.444091 |   2.45008  | 1.19146  | 0.237774   | 0.996098  |       0.760786  |   0.110014  |       5 |      3.55088 |
| MMP13  | monocyte-derived dendritic cell                         |      1.44072 |            1 |       8.1314e-05  |          1.22492e-06 |               8398 | 0.000121971 | 0.529119 | 0.444091 |   2.45008  | 1.19146  | 0.237774   | 0.996098  |       0.760786  |   0.110014  |       5 |      3.55088 |
| PLAU   | monocyte-derived dendritic cell                         |      1.66157 |           93 |       0.00756221  |          0.000113917 |               8398 | 0.174851    | 0.529119 | 0.444091 |   2.45008  | 1.19146  | 0.237774   | 0.996098  |       0.760786  |   0.110014  |       5 |      3.55088 |
| TXNIP  | monocyte-derived dendritic cell                         |      1.79998 |         1787 |       0.145308    |          0.00218893  |               8398 | 0.527893    | 0.529119 | 0.444091 |   2.45008  | 1.19146  | 0.237774   | 0.996098  |       0.760786  |   0.110014  |       5 |      3.55088 |

In [ ]:
results[0]['cell'].unique()[:5]

In [ ]:
ex_df = results[0].copy()
unique_cells = ex_df['cell'].unique()[:10]

ex_df = ex_df[ex_df['cell'].isin(unique_cells)][['cell', 'cell score']].drop_duplicates().reset_index(drop=True)
ex_df

In [ ]:
results[0][results[0]['cell']=='epithelial cell'][['gene', 'expression', 'cell count', 'cell percentage', 'tissue composition', 'score']]

In [ ]:
def export_dataframe(df, n=10):
    unique_cells = df['cell'].unique()[:n]
    filtered_df = df[df['cell'].isin(unique_cells)]

    final_df = (
        filtered_df.groupby('cell', as_index=False)
        .agg({'gene': lambda x: ", ".join(map(str, set(x))), 
              'cell score': lambda x: ", ".join(map(str, set(x)))})
    ).sort_values(by=['cell score'], ascending=[False])

    return final_df

In [ ]:
final_df = export_dataframe(results[0])
display_dataframe_as_markdown(final_df)

In [ ]:
def get_top3_cells(df):
    return df['cell'].unique()[:3]

In [ ]:
def export_top3_cells(results):

    top3cells = pd.DataFrame()
    for cluster in results:
        top3cells[f'{cluster}'] = get_top3_cells(results[cluster])

    top3cellsT = top3cells.T.reset_index()
    top3cellsT.columns = ['Cluster'] + [f'Cell {i+1}' for i in range(top3cellsT.shape[1]-1)]

    return top3cellsT

In [ ]:
display_dataframe_as_markdown(export_top3_cells(results))

# Validation with Tavily and Gemini Connected to a Database

In [48]:
import psycopg2
import os
from openai import OpenAI
from tavily import TavilyClient
from dotenv import load_dotenv

In [49]:
# Load environment variables
load_dotenv(override=True)

# Initialize Tavily and Gemini clients
tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [50]:
# Connect to PostgreSQL
def get_db_connection():
    return psycopg2.connect(os.getenv("DATABASE_URL"))

In [51]:
def check_database(tissue_name, cell_name, gene_name):
    query = """
        SELECT relationcertainty FROM gene_expression
        WHERE uberon = (
            SELECT uberon 
            FROM tissue 
            WHERE tissuename = %s
        )
        AND cl = (
            SELECT cl 
            FROM cell 
            WHERE cellname = %s
        )
        AND ensembl = (
            SELECT ensembl 
            FROM gene 
            WHERE genename = %s
        )
    """
    try:
        with get_db_connection() as conn:
            with conn.cursor() as cur:
                cur.execute(query, (tissue_name, cell_name, gene_name))
                result = cur.fetchone()
                return result[0] if result else None
    except Exception as e:
        print(f"Database error: {e}")
        return None

In [52]:
# Function to query Tavily using SDK
def query_tavily(tissue_name, cell_name, gene_name):
    query = f"Does {cell_name} express {gene_name} in {tissue_name}?"
    try:
        response = tavily_client.search(query= query, search_depth= "basic")
        sources = [result["url"] for result in response.get("results", [])]
        return sources if sources else None
    except Exception as e:
        print(f"Tavily Error: {e}")
        return None

In [53]:
# Function to summarize using OpenAI
def summarize_with_openai(tissue_name, cell_name, gene_name, sources):
    prompt = f"""
    Given the following sources, determine the confidence level that {cell_name} expresses {gene_name} in {tissue_name}.
    Return one of these values in valid JSON format:
    {{"status": 1.0}} → Yes
    {{"status": 0.67}} → Mostly Yes
    {{"status": 0.33}} → Mostly No
    {{"status": 0.0}} → No
    never return with markdown formatting.
    Sources: {sources}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": "You are an expert in cellular biology."},
                      {"role": "user", "content": prompt}]
        )
        output = response.choices[0].message.content
        return json.loads(output)["status"]
    except Exception as e:
        print(f"OpenAI Error: {e}")
        return None

In [54]:
def insert_into_database(tissue_name, cell_name, gene_name, source_urls, certainty):
    conn = psycopg2.connect(os.getenv("DATABASE_URL"))
    cur = conn.cursor()

    try:
        # Lookup IDs
        cur.execute("SELECT uberon FROM tissue WHERE tissuename = %s", (tissue_name,))
        uberon = cur.fetchone()
        if not uberon:
            raise ValueError(f"Tissue '{tissue_name}' not found.")
        uberon = uberon[0]

        cur.execute("SELECT cl FROM cell WHERE cellname = %s", (cell_name,))
        cl = cur.fetchone()
        if not cl:
            raise ValueError(f"Cell '{cell_name}' not found.")
        cl = cl[0]

        cur.execute("SELECT ensembl FROM gene WHERE genename = %s", (gene_name,))
        ensembl = cur.fetchone()
        if not ensembl:
            raise ValueError(f"Gene '{gene_name}' not found.")
        ensembl = ensembl[0]

        # Insert or update gene_expression
        cur.execute("""
            INSERT INTO gene_expression (uberon, cl, ensembl, relationcertainty)
            VALUES (%s, %s, %s, %s)
            ON CONFLICT (uberon, cl, ensembl)
            DO UPDATE SET relationcertainty = EXCLUDED.relationcertainty
            RETURNING expressionid
        """, (uberon, cl, ensembl, certainty))
        expressionid = cur.fetchone()[0]

        # Insert sources and validate links
        for url in source_urls:
            # Check or insert source
            cur.execute("SELECT sourceid FROM source WHERE url = %s", (url,))
            row = cur.fetchone()
            if row:
                sourceid = row[0]
            else:
                cur.execute("INSERT INTO source (url) VALUES (%s) RETURNING sourceid", (url,))
                sourceid = cur.fetchone()[0]

            # Insert validate link if not exists
            cur.execute("""
                INSERT INTO validate (expressionid, sourceid)
                VALUES (%s, %s)
                ON CONFLICT DO NOTHING
            """, (expressionid, sourceid))

        conn.commit()
        print(f"✅ Inserted/Updated expressionid {expressionid} with sources.")

    except Exception as e:
        conn.rollback()
        print(f"❌ Error: {e}")

    finally:
        cur.close()
        conn.close()

In [55]:
def validate_cell_gene_relation(tissue_name, cell_name, gene_name):
    print(f"\n[Validation Start] Processing: {tissue_name} - {cell_name} - {gene_name}")

    # Step 2: Check database first
    stored_status = check_database(tissue_name, cell_name, gene_name)
    if stored_status is not None:
        print(f"[Validation] Relation already exists in DB with status {stored_status}. Skipping process.")
        return float(stored_status)  # Ensure status is returned as a float

    # Step 3: Query Tavily
    try:
        sources = query_tavily(tissue_name, cell_name, gene_name)
    except Exception as e:
        print(f"Tavily Error: {e}")
        return None

    # Step 3a: If no sources found, set status to 0.0 and insert into database
    if not sources:
        status = 0.0
        print(f"[Validation] No references found in Tavily. Setting status to {status}.")
        insert_into_database(tissue_name, cell_name, gene_name, [], status)
        return float(status)  # Ensure status is a float

    # Step 4: Summarize with OpenAI
    status = summarize_with_openai(tissue_name, cell_name, gene_name, sources)
    status = float(status)  # Ensure status is a float
    print(f"[Validation] OpenAI summary status: {status}")

    # Step 5: Insert into Database
    insert_into_database(tissue_name, cell_name, gene_name, sources, status)
    print(f"[Validation] Inserted/Updated in DB with status: {status}")

    return status  # Return the validation score as a float

In [56]:
validate_cell_gene_relation("skin of body", "neural cell", "MMP3")


[Validation Start] Processing: skin of body - neural cell - MMP3
[Validation] Relation already exists in DB with status 0.33. Skipping process.


0.33

In [ ]:
x_df = results.copy()
for cluster in x_df:
    unique_cells = x_df[cluster]['cell'].unique()[:10]
    x_df[cluster] = x_df[cluster][['gene', 'cell', 'cell score']][x_df[cluster]['cell'].isin(unique_cells)]

In [ ]:
display_dataframe_as_markdown(x_df[0])

In [57]:
def main_data_validation(results, target_uberon_id):
    ex_df = results.copy()
    for cluster in ex_df:
        unique_cells = ex_df[cluster]['cell'].unique()[:10]
        ex_df[cluster] = ex_df[cluster][['gene', 'cell', 'cell score']][ex_df[cluster]['cell'].isin(unique_cells)]
    tissue_name = ontl.tissue(target_uberon_id)

    for cluster in ex_df:
        ex_df[cluster]["certainty score"] = ex_df[cluster].apply(lambda row: validate_cell_gene_relation(tissue_name, row["cell"], row["gene"]), axis=1)

    return ex_df

In [58]:
res = main_data_validation(results, TARGET_UBERON_ID)


[Validation Start] Processing: skin of body - epithelial cell - LGALS3BP
[Validation] Relation already exists in DB with status 0.67. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - RBM42
[Validation] Relation already exists in DB with status 0.33. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - GAMT
[Validation] Relation already exists in DB with status 0.33. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - IFI27
[Validation] Relation already exists in DB with status 1.0. Skipping process.

[Validation Start] Processing: skin of body - epithelial cell - KRT14
[Validation] Relation already exists in DB with status 1.0. Skipping process.

[Validation Start] Processing: skin of body - squamous epithelial cell - LGALS3BP
[Validation] Relation already exists in DB with status 0.67. Skipping process.

[Validation Start] Processing: skin of body - squamous epithelial cell - RBM42
[Validat

KeyboardInterrupt: 

In [ ]:
valid_df = res[0].copy()
mean_certainty = valid_df.groupby('cell')['certainty score'].mean().reset_index().rename(columns={'certainty score': 'cell certainty'})
final_df = valid_df.merge(mean_certainty, on='cell')


unique_cells = final_df['cell'].unique()
final_df = final_df[['cell', 'cell score', 'cell certainty']][final_df['cell'].isin(unique_cells)].drop_duplicates().reset_index(drop=True)



In [ ]:
def export_validation_dataframe(results):
    ex_df = results.copy()
    for cluster in ex_df:
        mean_certainty = ex_df[cluster].groupby('cell')['certainty score'].mean().reset_index().rename(columns={'certainty score': 'cell certainty'})
        ex_df[cluster] = ex_df[cluster].merge(mean_certainty, on='cell')

        unique_cells = ex_df[cluster]['cell'].unique()[:10]
        ex_df[cluster] = ex_df[cluster][['cell', 'cell score', 'cell certainty']][ex_df[cluster]['cell'].isin(unique_cells)].drop_duplicates().reset_index(drop=True)
    return ex_df

In [ ]:
tes = export_validation_dataframe(res)[0]

In [ ]:
display_dataframe_as_markdown(tes)

In [ ]:
display_dataframe_as_markdown(final_df)

In [ ]:
display_dataframe_as_markdown(mean_certainty)

In [ ]:
for cluster in results:
    valids = {}
    print(f"\nProcessing Cluster: {cluster}")
    valids[cluster] = data_validation(results[cluster], tl.tl_uberon(TARGET_UBERON_ID)).reset_index(drop=True)

In [ ]:
display_dataframe_as_markdown(results[0])

# Analysis using OpenAI and Tavily

In [ ]:
from langchain_community.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain_core.runnables import RunnableConfig, chain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import chain
from tavily import TavilyClient
import ast
import asyncio

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini", 
    temperature=0
)

In [ ]:
tavily = TavilyClient()

In [ ]:
from langchain_community.tools import TavilySearchResults
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain

In [ ]:
tool = TavilySearchResults(
    max_results=10,
    search_depth="basic",
    include_answer=True,
    include_raw_content=False,
    include_images=False,
)

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0.0
)

In [ ]:
res = tool.invoke('is epithelial cell of skin of body tissue related to these genes: [COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1]?')

In [ ]:
llm.invoke(f"answer with yes or no. based on this search results,\n{res}\n, is epithelial cell of skin of body tissue related to these genes: [COL17A1, EFNA1, GAMT, HSPA1A, IFI27, KRT14, LGALS3BP, RBM42, SCPEP1]?")

In [ ]:
tissue = tl.tl_uberon(TARGET_UBERON_ID)
tissue

In [ ]:
cell = final_df['cell'][0]
cell

In [ ]:
genes = [final_df['gene'][0]]
genes

In [ ]:
query = f"is {cell} of {tissue} tissue related to these genes: {genes}?"

In [ ]:
prompt = ChatPromptTemplate(query)

In [ ]:
prompt = ChatPromptTemplate(
    [
        ("system", "You are an expert in cellular biology. answer with a boolean array"),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

In [ ]:
final_df = export_dataframe(results[0])
display_dataframe_as_markdown(final_df)

In [ ]:
final_df.head(1)

In [ ]:
search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search)

# initialize the agent
agent = create_conversational_retrieval_agent(
    llm,
    tools=[tavily_tool],
)

In [ ]:
def generate_queries(cell_name, gene_names: list):
    return f"answer with yes or no. Is {cell_name} related to this genes: {gene_names}?"

In [ ]:
cell_names = results[0]['cell'].unique()[:20].tolist()

In [ ]:
gene_names = results[0][results[0]['cell'] == cell_names[1]]['gene'].tolist()

In [ ]:
gene_names

In [ ]:
cell_names

In [ ]:
test = generate_queries(cell_names[1], gene_names)
test

In [ ]:
# run the agent
resp = agent.invoke(test)

In [ ]:
resp

In [ ]:
resp

# Analysis using LangChain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_core.output_parsers import JsonOutputParser

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

load_dotenv(override=True)

client = MongoClient(os.getenv("MONGO_URI"), server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

In [ ]:
llm = ChatOpenAI(
    model="gpt-4o-mini-2024-07-18",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
#JSON Output formatter  
class CellAnnotation(BaseModel):
    class cell(BaseModel):
        cell_type : str = Field(description="name of the chosen cell.")
        reason : str = Field(description="explain the reasoning for choosing the cell. give comments about parameters from the data that support the choice.")
        
    explanation : list[str] = Field(description="explanation of your reasoning step by step. focus on the parameters and number of genes expressed that support the choice of the cell.")
    cells : list[cell]
  

In [ ]:
initial_df = df

In [ ]:
prompt = ChatPromptTemplate.from_template("""
                                          
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 5 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:

Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Given this dataset, outline the steps to create and apply a scoring system, then finally return a table of top 5 possible cell type identities for the cluster, and its calculated value.

Ensure that:
1. Describe each row of the dataset, and what it represents.
2. What metric is being used, and what value do they indicate.
3. The steps for scoring each possible cell type.
4. How to obtain the top five possible candidates.                                      
""")

chain = prompt | llm | StrOutputParser()

In [ ]:
parser = JsonOutputParser(pydantic_object=CellAnnotation)

prompt = PromptTemplate(
    template="""You are an expert in single-cell RNA sequencing.

        I have a cluster of cells with its genes as follow

        {initial_df}

        with the gene expression of each cell

        {expression_data}

        Based on the given data, which cell is the most valid to be annotated for this cluster of cells according to you?
        
        \n{format_instructions}""",
    input_variables=["initial_df", "expression_data"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | llm | parser

In [ ]:
response = chain.invoke({'initial_df': initial_df, 'expression_data': results[0].to_csv(sep='\t', index=False)})

In [ ]:
response

# Analysis using OpenAI

In [ ]:
# MODEL = "o3-mini"

In [ ]:
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [ ]:
initial_df = df

In [ ]:
instructions = """
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 5 most similar cell types to a given, unannotated cell cluster, based on a limited dataset. You will do this by creating and applying a scoring system.

The dataset is structured as follows:

Each row represents the cells that strongly express a top genes from the cluster. The columns are: "Gene" (a top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Follow these steps precisely to prepare the information, which will be automatically formatted into JSON:

**Step 1: Data Understanding**

*   Explain what each row in the dataset represents. Specifically, explain that each row shows a potential connection between the top gene from the cluster and one known cell type, and the metrics for THAT cell type, not for the cluster itself.
*   Explain the meaning of each column: "Gene", "Cell Name", "Expression Value", "Cell Percentage", "Cell Count", and "Tissue Composition." For each, explain what a higher or lower value might indicate *in the context of identifying similarity to the unannotated cluster*.

**Step 2: Develop a Scoring Function (CellTypeScore)**

*   Explain that the goal is to assign each known "Cell Name" (cell type) a score that reflects its similarity to the unannotated cluster.
*   Propose a formula for calculating a `GeneScore` for each *Gene* within each *Cell Name*. This formula should incorporate both "Cell Percentage" and "Expression Value". Justify why you chose to include these two metrics and how they are combined.
*   Propose a formula for combining the `GeneScore` values across all the `Gene` entries for a given "Cell Name" to calculate a final overall `CellTypeScore` for that cell type.

**Step 3: Calculate CellTypeScore for All Cell Names**

*   Explain that the goal is to now apply your scoring system to the known "Cell Name" values and find out which ones are the highest, with relation to what each Gene expresses.
*   Describe in precise steps what it should do: go through each unique "Cell Name" in the dataset, calculate a `GeneScore` and `CellTypeScore` based on Gene, Expression Value, and the percentage that are expressed.

**Step 4: Identify Top 5 Candidates**

*   Explain how to identify the top 5 candidates from the `CellTypeScore` values, now that they have been calculated in Step 3.
"""

In [ ]:
instructions ="""
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 3 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:

Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).

Given this dataset, analyze the dataset to determine similarity, then finally return a list of top possible cell type identities for the cluster.

Ensure that:
1. Never make any assumption. If a gene is not expressed, it should be treated as such and not used for consideration.
2. Understand each row of the data and what it represents.
3. Prioritize cell types that's expressing more genes from the cluster, then consider the cell count, cell percentage, and expression value.
4. The steps for determining each possible cell type.
5. How to obtain the top three possible candidates.
6. give the explanation as if writing a report
"""

In [ ]:
instructions ="""
You are an expert in analyzing single-cell RNA sequencing (scRNA-seq) data. Your goal is to identify the top 3 most similar cell types to a given, unannotated cell cluster, based on a limited dataset.

The dataset is structured as follows:
Each row represents a known cell type expressing top genes from a cluster. The columns are: "Gene" (top gene expressed in the cluster), "Cell Name" (the known cell type expressing the gene), "Expression Value" (level of expression), "Cell Percentage" (percentage of cells expressing the gene), "Cell Count" (number of cells), and "Tissue Composition" (percentage of cells in the tissue).
Given this dataset, analyze the dataset to determine similarity, then finally return a list of top possible cell type identities for the cluster. The dataset does not contain any representation of the unannotated cell cluster.

Cell similiarity is determined by the number of genes expressed in the cell then the cell count, cell percentage, and expression value. The more genes expressed in a cell, the higher the cell count, cell percentage, and expression value, the more similar the cell is to the unannotated cell cluster. the number of genes expressed in the cell is the most important factor in determining the similarity of the cell to the unannotated cell cluster.

Follow these steps precisely to prepare the information
Step 1 : Read the whole dataset
Step 2 : Explain to me what do you think about each cell's similiarity to the unannotated cell cluster based on the given data.
Step 3 : Based on you analysis, what are the top 3 possible cell type identities for the cluster?

Ensure that:
1. Never make any assumption. If a gene is not expressed, it should be treated as such and not used for consideration.
2. Do not consider cells that expressing lesser amount of genes from the cluster.
3. Do not consider cells that express a gene with parameters that deviate too much from other genes expressed by that cell.
4. Explanations must be written like a report.
"""

In [ ]:
#JSON Output formatter  
class CellAnnotation(BaseModel):
    class top_cell(BaseModel):
        class gene(BaseModel):
            gene_name : str = Field(description="Name of the gene expressed by the cell type")
            metrics : str = Field(description="cell count, cell percentage, and expression value of the gene")
        
        cell_type : str = Field(description="name of the chosen cell.")
        gene_info : list[gene]
        reason : str = Field(description="explain the reasoning for choosing the cell. give comments about parameters from the data that support the choice.")
    
    explanation : list[str] = Field(description="explanation of your reasoning step by step. focus on the parameters and number of genes expressed that support the choice of the cell.")
    cells : list[top_cell]

In [ ]:
print(results[0].to_csv(sep='\t', index=False))

In [ ]:
content = f"expression data : {results[23].drop(columns=['tissue composition']).to_csv(sep='\t', index=False)}"

In [ ]:
#normal
completion = client.beta.chat.completions.parse(
    model = 'gpt-4o-mini-2024-07-18',
    messages=[
        {"role": "developer", 
         "content": instructions},
        {"role": "user", 
         "content": content}
    ],
    temperature=0.0,
    response_format=CellAnnotation,
)

In [ ]:
#Reasoning Model
response = client.chat.completions.create(
    model = "o3-mini",
    reasoning_effort="medium",
    messages=[
        {"role": "developer", 
         "content": """
            - You are an expert in single-cell RNA sequencing. 
            - You will be provided with cell clustering data and gene expression data for each cell. 
            - Based on the given data, you need to identify the 4 most valid cell to be annotated for the cluster. 
            - Provide a list of cells in the order of validity and explain your reasoning step by step. only give out the name of the cells from the given data.
            - Analyze only the data provided. Do not reference or infer from any external sources
            - the answer is in the format of 
                {
                cells : [cell1, cell2, cell3, cell4],
                explanation : <explanation>
                }
            """},
        {"role": "user", 
         "content": content}
    ],
)

In [ ]:
print(completion.choices[0].message.parsed.model_dump_json(indent=4))

In [ ]:
import json

output_dict = json.loads(completion.choices[0].message.parsed.model_dump_json(indent=4))

In [ ]:
output_dict['cells']

# Color Test

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Sample DataFrame
data = {'Values': [5, 3, 8, 2, 9, 1, 7, 4, 6, 5, 3, 2]}
df = pd.DataFrame(data)

# Define normalization (scale from min=1 to max=9)
norm = mcolors.Normalize(vmin=1, vmax=9)
# Choose a colormap (reversed for light→dark)
cmap = plt.get_cmap('YlGnBu_r')  # Or 'Blues_r', 'Greys_r', etc.

# Map values to hex color codes
df['Color'] = df['Values'].apply(
    lambda x: mcolors.to_hex(cmap(norm(x)))  # Convert to hex
)

# Display DataFrame with hex color codes
print(df)

# Test Marker

In [ ]:
import pandas as pd
import requests
def get_marker(tissue, cell):
   # URL to send the POST request to
    url = "https://api.cellxgene.cziscience.com/wmg/v2/markers"

    # Payload to send in the POST request
    payload = {
        "celltype": cell,
        "n_markers":50,
        "organism":"NCBITaxon:9606",
        "test":"ttest",
        "tissue":tissue
    }

    # Send POST request
    return requests.post(url, json=payload).json()['marker_genes']

In [ ]:
get_marker("UBERON:0001043", "cell CL:1000615")

# Test Function 2

In [ ]:
resultsv2 = main_data_analysisv2(ensembl_df, TARGET_UBERON_ID)

In [ ]:
display_dataframe_as_markdown(resultsv2[23])